In [ ]:
# default_exp model.cvae
#default_cls_lvl 2

In [ ]:
#hide 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# export
from snkrfinder.imports import *
from snkrfinder.core import *
from snkrfinder.data.munge import *
from snkrfinder.model.core import *
#from snkrfinder.model.transfer import *

from fastai.test_utils import show_install, synth_learner, nvidia_smi, nvidia_mem

ModuleNotFoundError: No module named 'snkrfinder'

In [ ]:
#hide
from nbdev.showdoc import *

# first snkrfinder.model.cvae

## OVERVIEW: cvae module - convolutional variational auto encoder

> TODO: clean up this preamble section

preamble: This is a project initiated while an Insight Data Science fellow.  It grew out of my interest in making data driven tools in the fashion/retail space I had most recently been working.   The original over-scoped idea was to make a shoe desighn tool which could quickly develop some initial sneakers based on choosing some examples, and some text descriptors.  Designs are constrained by the "latent space" defined (discovered?) by a database of shoe images.  However, given the 3 week sprint allowed for development, I pared the tool down to a simple "aesthetic" recommender for sneakers, using the same idea of utilizing an embedding space defined by the database fo shoe images.


Most of the code is derived from the work of @EtieeneT. (e.g.: TabularData https://github.com/EtienneT/TabularVAE and later https://github.com/EtienneT/vae )


### sneaker image encoding via various flavors of auto-encoder.  
- Auto Encoder: AE
    - ~~linear encoder / decoder~~
    - convolutional encoder / decoder
        - encoders
            - pretrained (e.g. resnet, mobilenet_v2)
            - 'vanilla' convolutional
            - 'vanilla' ResBlock

        - decoders
            - 'vanilla' convolutional
            - 'vanilla' ResBlock
    - Linear Latent representation (`LatentLayer`)

- Variational- Auto Encoder:  VAE or 𝜷-VAE      
    - Variational "bottleneck": "reparameterazation trick" for two variable linear latents
        - 𝜷-VAE: by increasing the relative strenght of error signal of the KLD through a beta variational parameter we can get better formed latent spaces which are somewhat "disentangled". That is have some semantic or meaningful structure in the representation. 
    - MMD- variational autoencoder uses a Linear Latent and MMD as the retularizer
        - The KLD has some problems and "over-regularizes" or simply isn't strong enough compared to the Maximum Mean Discrepancy (MMD) loss and tends to converge to a somewhat degenerate solution
        - MD regularization of the latent space has great advantage but at a computational expense



### FUTURE EXTENSIONS:
    -  model based data cleaning (widget module?)
    - GAN finetuning?
    - crappify general pattern
    - throw out based on inspection of high loss
    - try mixed labels?  things that are >50% sneakers included???
    
    
### TODO: fix loss / metrics reduction logic
batchmean should *only* apply to KLD because the mean over latents in not meaningful.
'none', 'mean', 'sum' for reduction over "batches"

all the other _scaling_ should come through alpha....


### TODO: find bugs in github actions

In [ ]:
#hide
show_install()

In [ ]:
#hide
#nvidia_smi()


In [ ]:
#hide 
# fix our base directory
print(Path().cwd())
os.chdir(L_ROOT)
print(Path().cwd())

## Using  fastai V2: data pipelining
- Datablock API


Load the saved merged database, and set the seeds.  And doublecheck our data is where we expect.

In [ ]:
#hide
df = pd.read_pickle(f"data/{COMBINED_DF}.pkl")
np.random.seed(3333)
torch.manual_seed(3333)


In [ ]:
#hide
#TODO:  fix the data layout... the links and paths are screwy
image_path = L_ROOT/"data"
#image_path = D_ROOT/DBS['zappos']
batch_size = 64

L([image_path/d for d in df.path])

In [ ]:
# export

def prep_df_for_datablocks(df):
    df = df[["path","train","test","validate","t_t_v","Category"]].copy()
    # I could remove all the "test" rows... for now i'll choose an alternate strategy:
    # Drop all the "test" rows for now, and create an "is_valid" column...
    # should probably drop a ton of columns to jus tkeep the file paths...
    # just keep what we'll need below
    df.loc[:,'is_valid'] = df.test | df.validate
    df.loc[:,'og_idx'] = df.index

    return df



In [ ]:
df = prep_df_for_datablocks(df)

In [ ]:
#export
def get_ae_btfms(stats = 'sneaker'):
    # could use globals IM_STATS['sneaker'] and IM_STATS['imagenet']
    im_stats = ([.5,.5,.5],[.5,.5,.5]) if stats == 'sneaker' else imagenet_stats
    batch_tfms = Normalize.from_stats(*im_stats)
    #batch_tfms = Normalize.from_stats([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    rand_tfms = aug_transforms(mult=1.0, 
               do_flip=True, 
               flip_vert=False, 
               max_rotate=5.0, 
               min_zoom=.95, 
               max_zoom=1.0, 
               max_lighting=0.1, 
               max_warp=0.1, 
               p_affine=0.66, 
               p_lighting=0.2, 
               xtra_tfms=None, 
               size=None, 
               mode='bilinear', 
               pad_mode='border', 
               align_corners=True, 
               batch=False, 
               min_scale=1.0)
    return rand_tfms+[batch_tfms]

def get_ae_no_aug(stats = 'sneaker'):
    im_stats = ([.5,.5,.5],[.5,.5,.5]) if stats == 'sneaker' else imagenet_stats
    batch_tfms = Normalize.from_stats(*im_stats)
    return [batch_tfms]

## FastAI "block" for autoencoder

Next we need our own version of ReadTabBatch that will return our inputs


In [ ]:
#export

# NO CLUE WHY WE NEED TO HAVE THIS.... copied
class TensorPoint(TensorBase):
    "Basic type for points in an image"
    _show_args = dict(s=10, marker='.', c='r')

    @classmethod
    def create(cls, t, img_size=None)->None:
        "Convert an array or a list of points `t` to a `Tensor`"
        return cls(tensor(t).view(-1, 2).float(), img_size=img_size)

    def show(self, ctx=None, **kwargs):
        if 'figsize' in kwargs: del kwargs['figsize']
        x = self.view(-1,2)
        ctx.scatter(x[:, 0], x[:, 1], **{**self._show_args, **kwargs})
        return ctx


class Tensor2Vect(TensorPoint): pass
# TODO:  instantiate a show method


class LatentsTensor(Tensor2Vect):
    "Basic type for latents as Tensor inheriting from TensorPoint (vectors)"
    @classmethod
    def create(cls, ts, img_size=IMG_SIZE): 
        "create IMG_SIZE attr to register plotting..."

        if isinstance(ts,tuple):
            mu,logvar = ts
        elif ts is None:
            mu,logvar = None,None
        else:
            mu = None
            logvar = None
        if mu is None: mu = torch.empty(0)
        elif not isinstance(mu, Tensor): Tensor(mu) 
        
        if logvar is None: logvar = torch.empty(0)
        elif not isinstance(logvar,Tensor): Tensor(logvar)
        
        t = torch.cat([mu,logvar],dim=-1) # in case its a batch?
                
        return cls(tensor(t).view(-1, 2).float(), img_size=img_size)

    #     def show(self, ctx=None, **kwargs):
    #         if 'figsize' in kwargs: del kwargs['figsize']
    #         x = self.view(-1,2)
    #         ctx.scatter(x[:, 0], x[:, 1], **{**self._show_args, **kwargs})
    #         return ctx
    #         mu,logvar = self
    #         if not isinstance(mu, Tensor) or not isinstance(logvar,Tensor): return ctx

    #         title_str = f"mu-> {mu.mean():e}, {mu.std():e}  logvar->{logvar.mean():e}, {logvar.std():e}"

    #         if 'figsize' in kwargs: del kwargs['figsize']
    #         if 'title' in kwargs: kwargs['title']=title_str
    #         if ctx is None:
    #             _,axs = plt.subplots(1,2, figsize=(12,6))
    #             x=torch.linspace(0,1,mu[0].shape[0])
    #             axs[0].scatter(x, mu[:], **{**self._show_args, **kwargs})
    #             axs[1].scatter(x, logvar[:], **{**self._show_args, **kwargs})
    #             ctx = axs[1]

    #         ctx.scatter(mu[:], logvar[:], **{**self._show_args, **kwargs})
    #         return ctx


In [ ]:
#export

def df_get_x(r): 
    "datablock df helper for VAE Block using `LatentTuple`"
    return image_path/r['path']

def df_get_y(r): 
    "datablock df helper for VAE Block using `LatentTuple`"
    return (df_get_x(r),None,None)

In [ ]:
#export 


def LatentsTensorBlock(): 
    "Class wrapper for the AE `LatentTensor` Block"
    return TransformBlock(type_tfms=LatentsTensor.create, batch_tfms=noop)


def df_ae_x(r,im_path=L_ROOT/"data"): 
    "Autoencoder LatentsTensorBlock datablock df helper"
    return im_path/r['path']


# need to make sure that we get the image whihc is "Identical" to the input.. how to test?

def df_ae_y(r): 
    "The target is the same as the input for AE"# lambda o: o
    return df_ae_x(r)



#export 
# could we do a typedispatch to manage the transforms...?
# def VAETargetTupleBlock(): 
#     return TransformBlock(type_tfms=VAETargetTuple.create, batch_tfms=IntToFloatTensor)

def LatentTupleBlock(): 
    "Class wrapper for the AE `LatentTuple` Block (depricated)"
    return TransformBlock(type_tfms=LatentTuple.create, batch_tfms=noop)



Don't forget to set  `n_inp=1`. Otherwise the default to make the  input to 1-len(blocks).  Also note that the `FeatsResize` is used to avoid the random jittering from resize during training.  Only the very narrow batch augmentations will be used.

In [ ]:
#export
# 

def get_ae_DataBlock(aug=True,im_path=L_ROOT/"data",stats = 'sneaker',im_size=IMG_SIZE):
    "wrapper to get the standard AE datablock with `ImageBlock`,`LatentTensor` target"
    # use partials or a class wrapper to get around this yucky hack
    global image_path
    image_path = im_path
    
    mytfms = get_ae_btfms(stats=stats) if aug else get_ae_no_aug(stats=stats)
    block = DataBlock(blocks=(ImageBlock(cls=PILImage), ImageBlock(cls=PILImage), LatentsTensorBlock ),
              get_x=df_ae_x, 
              get_y=[df_ae_y, noop], #don't need to get the LatentsTensorBlock, just create
              splitter=ColSplitter('is_valid'), 
              item_tfms= FeatsResize(im_size,method='pad', pad_mode='border'),
              batch_tfms = mytfms,
              n_inp = 1)
 
    return block



In [ ]:
# hide

# block = get_ae_DataBlock(aug=True)
# block.summary(df)


In [ ]:
# hide
# dls = block.dataloaders(df, batch_size=batch_size)

In [ ]:
#hide
# dls.show_batch()

## creating the VAE

Variational Auto-Encoder for fastai

I'm going to use a generic convolutional net as the basis of the encoder, and its reverse as the decoder. This is a proof of concept for using the _fastai_ framework, and will experiment with pre-trained resnet and MobileNet_v2 later. I'd like to use the MobileNet_v2 as a direct link ot the "SneakerFinder" tool which motivated this experiment. [see SneakerFinder]

A variational "module" will sit between the encoder and decoder as the "Bottleneck". The Bottleneck will map the resnet features into a latent space (e.g. ~100 dimensions) represented of standard normal variables.  The "reparameterization trick" will sample from this space and the "decoder" will generate images. 

Finally a simple "decoder" will sample from the variational latents space and be trained to reconstruct the images. 

The intention is the latent space can be used to generate novel sneaker images.



### AE:  deterministic AutoEncoder ( _non- variational_)

Although we give up the original utility we are going for --  creating new sneakers via the latent space -- having otherwise equivalent non-variational autoencoders for reference will be great. Furthermore, this latent space representation will be amenable to a MMD regularization later on.  This will be useful to avoid some of the limitiations of the KLD as a regularizer (overestimation of variance, and some degenerate convergences).
Its sort of _hack_-ey but keeping the tooling equivalent to the betaVAE will ultimatel give some advantages.

In [ ]:
#export
class UpsampleBlock(Module):
    def __init__(self, up_in_c:int, final_div:bool=True, blur:bool=False, **kwargs):
        """
        Upsampling using PixelShuffle_INCR and ConvLayer
            - up_in_c :  "Upsample input channel"
        """
        self.shuf = PixelShuffle_ICNR(up_in_c, up_in_c//2, blur=blur, **kwargs)
        ni = up_in_c//2
        nf = ni if final_div else ni//2
        self.conv1 = ConvLayer(ni, nf, **kwargs) # since we'll apply it by hand...
        self.conv2 = ConvLayer(nf, nf, **kwargs)

    def forward(self, up_in:Tensor) -> Tensor:
        up_out = self.shuf(up_in)
        return self.conv2(self.conv1(up_out))
    



In [ ]:
#export

class LatentLayer(Module):
    """
    This layer encodes the latent "bottleneck" and is constructed to work with the specified VAE DataBlock be a replacement for 
    the variational (reparameter trick) layer for otherwise identical architecture
    """
    
    def __init__(self,in_features,latent_features):
        """ 
        Compose a linear latent layer such that the mechanics are equivalent to the VAE
              the "dummy" can be used for a shaddow logvar track a KLD estimate divergence 
              from latent gaussian prior
              compute the variance across batchs for each latent feature as the dummy_var
        """       
        self.latent = nn.Linear(in_features,latent_features)
    
    def forward(self,h):
        z = self.latent(h)
        #dummy_var = (z.var(dim=1).unsqueeze(-1).expand(z.size()) ) #variance across latent dim for each image
        dummy_var = (z.var(dim=0).unsqueeze(0).expand(z.size()) ) #latent variance across batch
        dummy_mu = z
        return z, dummy_mu, dummy_var.log()
        
        #return z, torch.zeros_like(z)
    

In [ ]:
#export

class AEEncoder(Module):
    def __init__(self,arch_body,enc_dim, hidden_dim=None, im_size=IMG_SIZE):
        """
        arch_body   list of layers (e.g. arch.children()[:cut])    
        enc_dim,
        hidden_dim. number of linear features to sandwich between the feature encoder and the latent layers
        """                
        arch = arch_body + [Flatten()]

        if hidden_dim:  # i.e. is not None
            arch += [nn.Linear(enc_dim,hidden_dim)]
                    # [LinBnDrop(enc_dim,hidden_dim,bn=True,p=0.0,act=nn.ReLU(),lin_first=True)]

        self.encoder = nn.Sequential(*arch)
        store_attr('enc_dim,hidden_dim')
        
    def forward(self, x):
        return self.encoder(x)



      
#### TODO: refactor this to take a "BLOCK" input so we can have either UpsampleBlocks or ResBlockUpsampleBlocks      
class AEDecoder(Module):
    def __init__(self, hidden_dim=None, latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE):
        """
        Decoder Module made of `UpsampleBlock`s returning the latent representation back into an "image"
            latent_dim - dimension of latent representation
            hidden_dim - optional additional linear layer between the latent and decoder
            im_size - passed to make sure we are scaling back to the right size
            out_range - ensures the output is on teh same scale as the _normalized_ input image
        """
        #decoder        
        n_blocks = 5              
        BASE = im_size//2**5   
        
        hidden = im_size*BASE*BASE if hidden_dim is None else hidden_dim
        z_fc = [nn.Linear(latent_dim,hidden)] # [LinBnDrop(latent_dim,hidden,bn=True,p=0.0,act=nn.ReLU(),lin_first=True)]
        if hidden_dim:  # i.e. is not None
            z_fc += [nn.Linear(hidden,im_size*BASE*BASE)]  # should the hidden layer have activationa and/or batchnorm?
            #z_fc += [LinBnDrop(hidden,im_size*n_blocks*n_blocks,bn=True,p=0.0,act=nn.ReLU(),lin_first=True)]
        
        
        nfs = [3] + [2**i*BASE for i in range(n_blocks+1)] 
        nfs.reverse()
        n = len(nfs)
        
        modules =  [UpsampleBlock(nfs[i]) for i in range(n - 2)]        
        self.decoder = nn.Sequential(*z_fc,
                                      ResizeBatch(im_size,BASE,BASE),
                                      *modules,
                                      ConvLayer(nfs[-2],nfs[-1],
                                                ks=1,padding=0, norm_type=None, #act_cls=nn.Sigmoid) )
                                                act_cls=partial(SigmoidRange, *out_range)))
        
        store_attr('latent_dim, hidden_dim,im_size,out_range')

    def forward(self, z):    
        return self.decoder(z)
        
        

It is convenient to avoid the class wrappers to simplify the param splitting for freezing the pre-trained arcitecture.  
We could enumerate the class layers and return sequential, but simply making some functions to put the layers togeter is better.

In [ ]:
# export
    
def build_AE_encoder(arch_body,enc_dim, hidden_dim=None, im_size=IMG_SIZE):
    "wrapper to sequential-ize AEEncoder class"
    encoder = AEEncoder(arch_body,enc_dim=enc_dim, hidden_dim=hidden_dim, im_size=im_size)
    return nn.Sequential(*list(encoder.children()))

    
        
def build_AE_decoder(hidden_dim=None, latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE):
    "wrapper to sequential-ize AEDecoder class"
    decoder = AEDecoder(hidden_dim=hidden_dim, latent_dim=latent_dim, im_size=im_size,out_range=out_range)
    return nn.Sequential(*list(decoder.children()))
    

    

In [ ]:
#export
        
class AE(Module):
    def __init__(self,enc_parts,hidden_dim=None, latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE):
       
        """
        inputs:  
            arch, cut,pretrained
            enc_dim
            latent_dim
            hidden_dim
            
        """
        enc_arch,enc_feats,name = enc_parts
        
        BASE = im_size//2**5
        enc_dim = enc_feats * BASE**2  # 2**(3*3) * (im_size//32)**2 #(output of resneet) #12800

        #encoder
        self.encoder = build_AE_encoder(enc_arch,enc_dim=enc_dim, hidden_dim=hidden_dim, im_size=im_size)

        in_dim = enc_dim if hidden_dim is None else hidden_dim
        
        # AE Bottleneck
        self.bn = LatentLayer(in_dim,latent_dim)     
        
        #decoder
        self.decoder = build_AE_decoder(hidden_dim=hidden_dim, latent_dim=latent_dim, im_size=im_size,out_range=out_range)

        store_attr('name,enc_dim, in_dim,hidden_dim,latent_dim,im_size,out_range') # do i need all these?
            
    
    def decode(self, z):    
        return self.decoder(z)
    
    def encode(self, x):
        h = self.encoder(x)
        return self.bn(h)
            
    
    def forward(self, x):
        #z, mu, logvar = self.encode(x)
        #         h = self.encoder(x)
        #         z, mu, logvar = self.bn(h) # reparam happens in the VAE layer
        #         x_hat = self.decoder(z)
        
        z,mu,logvar = self.encode(x)  # z and mu are the same for 
        x_hat = self.decode(z)
        latents = torch.stack([mu,logvar],dim=-1)
        
        return x_hat, latents # assume dims are [batch,latent_dim,concat_dim]

    

#### Loss

These inherit from `Module`. (fastai's wrapper on `nn.Module`).  The base class `AELoss` initializes wiht a `batchmean`, `alpha`, and `useL1` parameters to set how the loss will be aggregated and regularized.  For the basice AutoEncoder we'll regularize the latent with a L1 to keep the magnitudes from exploding.   

> NOTE: Here `batchmean` means we will divide the loss (either L1 or L2 depending on `useL1` flag) by which uses `reduction='sum'` by the batch size.   This technically makes it a _cost_ computed for each batch.   This same convention will be used later for KL-Divergence and MMD latent regularizers.

In [ ]:
# export

        
# class L1LatentReg(Module):
#     """
#      add alpha?
#     """
#     def __init__(self, batchmean=False):
#         """
#         reduction 'sum', else 'batchmean'
        
#         """
#         l_one = self._L1mean if batchmean else self._L1
#         store_attr('batchmean,l_one')
    
#     def _L1(self, a):
#         return a.abs().sum()
    
#     def _L1mean(self, a):
#         return a.abs().sum(dim=1).mean()

#     def forward(self,z):
#         return self.l_one(z)        
            
     
class AELoss(Module):
    """
    wrapper for loss_func which deals with potential annealed kl_weight
    does MSE with 'mean' reduction
    'batchmean'  averages as 'sum' MSE over batches
    simple L1 regularizer on latent dimension
    """
    def __init__(self, batchmean=False, alpha=1.0,useL1=False):
        """
        reduction 'sum'
        
        """
        pix_loss = MSELossFlat(reduction='sum') if not useL1 else L1LossFlat(reduction='sum')
        store_attr('pix_loss,alpha,batchmean')
        
    def l_one_reg(self,pix_dim,z):
        l_one = z.abs().sum()
        l_one *= (3*pix_dim*pix_dim)/z.size()[1] 
        return l_one
    
    def forward(self, preds, *target):
        """
        pred =(x_hat,KLD,kl_weight) #mu,log_var, kl_weight)
        target is x (original) 
        """        
        # this handles the annealed kl_weight and passing the mu,logvar around we added...
        if(len(preds) == 3):
            x_hat, latents, _ = preds
        else: #if len(preds) == 2:  # we should never get here... unless we delete teh callback
            x_hat, latents = preds
            

        z, _ = latents.split(1,dim=2)
        bs = latents.size()[0]

        #note: both mse and l1_reg are summing errors over batches, and pixels or latents 
        pix_err = self.pix_loss(x_hat, target[0]) 
        pix_dim = x_hat.size()[-1]
        l1_reg = self.l_one_reg(pix_dim,z)
        
        total =  pix_err + self.alpha*l1_reg
        
        total *= (1./bs) if self.batchmean else 1.0
        
        return total

#### Metrics

These are called by the recorder callback and collect quantities to track training.  I made a simple meta class so I don't have to repreat the reset and mean value property for all the metrics.

In [ ]:
# export

class MyMetric(Metric):
    "meta-class for simple average over epoch metric quantities"
    def reset(self):
        "Clear all targs and preds"
        self.vals = []
    @property
    def value(self):
        return np.array(self.vals).mean()
    
class L1LatentReg(MyMetric):
    "Latent Regularizer with sum reduction and optinal batchmean scaling"
    def __init__(self,batchmean=False,alpha=1.0): 
        vals = []
        store_attr('vals,batchmean,alpha')
    
    def accumulate(self, learn):
        #         pix_dim = to_detach(learn.y[0].size()[-1])
        latents = to_detach(learn.pred[1])
        bs = latents.size()[0]
        z, _ = latents.split(1,dim=2)
        #nll = torch.abs(recon_x - x).mean()
        l_one = z.abs().sum()
        #         l_one *= (3*pix_dim*pix_dim)/z.size()[1] 
        l_one *= (self.alpha/bs) if self.batchmean else self.alpha

        self.vals.append(l_one)
                        
        

These are some helpers for computing the KL Divergence as a function and a `Module`

In [ ]:
# export

def KLD(mu,logvar):
    "KLD helper which sum across latents, but not batches"
    return -0.5 * torch.sum(1 + logvar - mu*mu - logvar.exp(),1)

        
class KLDiv(Module):
    """
    Module for computing the KL Divergence from a unit normal distribution.
    'batchmean' option sums first and averages over batches 
    """
    def __init__(self, batchmean=False):
        """
        reduction 'sum', else 'batchmean'
        
        """
        store_attr('batchmean')
        
    def __KLD(self,mu,logvar):
        "KLD helper which sum across latents, but not batches"
        return -0.5 * torch.sum(1 + logvar - mu*mu - logvar.exp(),1)


    def forward(self, mu, logvar):
        """
        pred =(x_hat,KLD,kl_weight) #mu,log_var, kl_weight)
        target is x (original) 
        """        
        kld = self.__KLD(mu,logvar)
        
        kld = kld.mean() if self.batchmean else kld.sum()
        return kld
    
    
        

In [ ]:
#export
    
class L2MeanMetric(MyMetric):
    "Mean square error"
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        x = to_detach(learn.y[0])
        recon_x = to_detach(learn.pred[0])
        nll = (recon_x - x).pow(2).mean()
        #nll = torch.mean((recon_x - x)**2)
        self.vals.append(nll)

class L1MeanMetric(MyMetric):
    "Mean absolute error"
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        x = to_detach(learn.y[0])
        recon_x = to_detach(learn.pred[0])
        #nll = torch.abs(recon_x - x).mean()
        nll = (recon_x - x).abs().mean()
        self.vals.append(nll)
        
class L2Metric(MyMetric):
    "Sum square error"
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        x = to_detach(learn.y[0])
        recon_x = to_detach(learn.pred[0])
        nll = (recon_x - x).pow(2).sum()
        #nll = torch.mean((recon_x - x)**2)
        self.vals.append(nll)

class L1Metric(MyMetric):
    "Sum absolute error"
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        x = to_detach(learn.y[0])
        recon_x = to_detach(learn.pred[0])
        #nll = torch.abs(recon_x - x).mean()
        nll = (recon_x - x).abs().sum()
        self.vals.append(nll)
        
        
class L2BMeanMetric(MyMetric):
    "Summed square error average across batch "
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        x = to_detach(learn.y[0])
        recon_x = to_detach(learn.pred[0])
        nll = (recon_x - x).pow(2).sum(dim=[1,2,3]).mean()
        #nll = torch.mean((recon_x - x)**2)
        self.vals.append(nll)

class L1BMeanMetric(MyMetric):
    "Summed abs error average across batch "
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        x = to_detach(learn.y[0])
        recon_x = to_detach(learn.pred[0])
        #nll = torch.abs(recon_x - x).mean()
        nll = (recon_x - x).abs().sum(dim=[1,2,3]).mean()
        self.vals.append(nll)
        
        
class KLWeightMetric(MyMetric):
    "Injected KLD weighting"
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        #kl = learn.model.kl_weight
        kl = learn.opt.hypers[0]['kl_weight']
        self.vals.append(to_detach(kl)) 
        
        
class RawKLDMetric(MyMetric):
    "KLD Metric, `batchmean` averages across batches"
    def __init__(self,batchmean=False): 
        vals = []
        _KLD = KLDiv(batchmean=batchmean)
        store_attr('vals,batchmean,_KLD')
        
    def accumulate(self, learn):
        latents = learn.pred[1]
        mu, logvar = latents.split(1,dim=2)
        kld = self._KLD(mu,logvar)
        self.vals.append(to_detach(kld)) 
        
class WeightedKLDMetric(MyMetric):
    """weighted KLD Metric, `batchmean` averages across batches
            the "effective" KLD regularization in e.g. a 𝜷-BAE
    """
    def __init__(self,batchmean=False,alpha=1.0): 
        vals = []
        _KLD = KLDiv(batchmean=batchmean)
        store_attr('vals,batchmean,alpha,_KLD')
        
    def accumulate(self, learn):      
        latents = learn.pred[1]
        mu, logvar = latents.split(1,dim=2)
        kld = self.alpha*self._KLD(mu,logvar)
        self.vals.append(to_detach(kld)) 
        #         latents = to_detach(learn.pred[1])
        #         mu, logvar = latents.split(1,dim=2)
        #         kld = _KLD(mu,logvar).mean() if self.batchmean else _KLD(mu,logvar).sum()
        #         self.vals.append(self.alpha*kld) 
        
        
class MuMetric(MyMetric):
    "average latent value (e.g. avg(`mu`)"
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        latents = to_detach(learn.pred[1])
        mu, logvar = latents.split(1,dim=2)
        self.vals.append(mu.mean())

    
class MuSDMetric(MyMetric):
    "standard deviation of latent 𝝁 value (e.g. std(`mu`) )"
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        latents = to_detach(learn.pred[1])
        mu, logvar = latents.split(1,dim=2)
        self.vals.append(mu.std())

    
class StdMetric(MyMetric):
    "average of latent 𝝈 value (e.g. std(exp(.5*`logvar`) )"
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        latents = learn.pred[1]
        mu, logvar = latents.split(1,dim=2)
        std = torch.exp(0.5 * logvar).mean()
        self.vals.append(to_detach(std)) 
        
class StdSDMetric(MyMetric):
    "standard deviation of latent 𝝈 value (e.g. std(exp(.5*`logvar`) )"
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        latents = learn.pred[1]
        mu, logvar = latents.split(1,dim=2)
        std = torch.exp(0.5 * logvar).std()
        self.vals.append(to_detach(std)) 
        
class LogvarMetric(MyMetric):
    "average of latent log(𝝈*𝝈) value (e.g. mean(`logvar`))"
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        latents = to_detach(learn.pred[1])
        mu, logvar = latents.split(1,dim=2)
        self.vals.append(logvar.mean())
                
class LogvarSDMetric(MyMetric):
    "standard deviation of latent log(𝝈*𝝈)  value (e.g. std(`logvar`)"
    def __init__(self): self.vals = []
    def accumulate(self, learn):
        latents = to_detach(learn.pred[1])
        mu, logvar = latents.split(1,dim=2)
        self.vals.append(logvar.std())





#### metrics helpers

In [ ]:
# export

def default_AE_metrics(alpha,batchmean,useL1):
    "long-ish default list of metrics for the AE"
    
    first = L2BMeanMetric() if batchmean else L2MeanMetric()
    second = L1BMeanMetric() if batchmean else L2MeanMetric()
    
    if useL1: first,second = second,first
        
    metrics = [first,
                L1LatentReg(batchmean=batchmean,alpha=alpha),
                MuMetric(), 
                StdMetric(),
                LogvarMetric(),
                second,
                WeightedKLDMetric(batchmean=batchmean,alpha=alpha),
                MuSDMetric(), 
                LogvarSDMetric(),              
               ]
    return metrics

def short_AE_metrics(alpha,batchmean,useL1):
    "short default list of metrics for the AE"
    
    first = L2BMeanMetric() if batchmean else L2MeanMetric()
    second = L1BMeanMetric() if batchmean else L2MeanMetric()
    
    if useL1: first,second = second,first
        
    metrics = [first,
                L1LatentReg(batchmean=batchmean,alpha=alpha),
                MuMetric(),              
               ]
    return metrics


### Callback - AnnealedLoss

The AnnealedLoss Callback basically injects a kl_weight parameter into the loss so we can start training without the full KLD regularization for the beta-VAE version.

The fastai `Learner` class does the training loop.  It took me a little digging into the code to figure out how Metrics are called since its not really stated anywhere in the documentation (_Note: create PR for fastai for extra documentation on `Metrics` logic_).  By default one of the key `Callbacks` is the `Recorder`.  It prints out the training summary at each epoch (via `ProgressCallBack`) and collects all the `Metrics`. Which by default only loss is a `train_met` and others are `valid_met`.
The `Recorder` resets (maps `reset()` to all mets) the metrics `before_train` and `before_valid`. The `Recorder` maps `accumulate()` to the metrics on `after_batch`.  Finally

AnnealedLossCallback will inject the latent mu and logvar and a kl_weight variable into our loss.  The `mu` and `logvar` will be used to compute the KLD. The kl_weight is a scheduled weighting for the KLD. You can see the schedule graph of the parameter. At the beginning it will be 0, thus the KLD part of the loss will get ignored. So during 10% of training, we will fit a normal auto-encoder. Then gradually for 30% of trainning, increase kl_weight to 1 and then remain there for the remaining training time so that the auto encoder now becomes full variational. The way this callback is done, the loss will receive this parameter, but not the model.

In [ ]:
#export
        
class AnnealedLossCallback(Callback):
    "injects `kl_weight` for access during loss function calculation"
    def after_pred(self):
        kl_weight = self.learn.pred[0].new(1)
        kl_weight[0] = self.opt.hypers[0]['kl_weight'] if 'kl_weight' in self.opt.hypers[0].keys() else 1.0
        self.learn.pred = self.learn.pred + (kl_weight,)
    def after_batch(self):
        pred, latents, _ = self.learn.pred
        self.learn.pred = (pred,latents)
        
        
def default_KL_anneal_in():
    "reasonable default for 'warming up' the KL Div"
    return combine_scheds([ .7, .3], [SchedCos(0,1), SchedNo(1,1)])

In [ ]:
# lets start with ~10 epochs of no KL regularizaiton, and ramp through the next 5
n_epochs = 10
f_init = combine_scheds([.1, .7, .2], [SchedNo(0,0),SchedCos(0,1), SchedNo(1,1)])
# f = combine_scheds([.8, .2], [SchedCos(0,0), SchedCos(0,.5)])
p = torch.linspace(0.,1.,100)
pp = torch.linspace(0.,1.*n_epochs,100)

plt.plot(pp,[f_init(o) for o in p])

> WARNING: Avoid using early stopping because the AnnealedLossCallback will make the loss grow once the KL divergence weight kicks in. 




I want to note something here that was a little confusing to me: `params(model)` is a builtin fastai `PyTorch.core` function which returns all of the parameters of the modules.    i.e.
    
```python    
def params(m):
    "Return all parameters of `m`"
    return [p for p in m.parameters()]
```


The toplevel `fastai core` functions with simple names that _almost_ match class attributes was one of my biggest stumbling blocks in getting acquainted with the fastai v2 API.  (The other is the documentation which is _autogenerated_ by the fastdev frameworks from their development noteboooks.  More on that struggle and my tips if that is troblesome for you later (here).

> NOTE: that it is crucial that you don't freeze the batch norm layers.   The `bn_splitter` collects out all the batchnorm layers.  The simple splitting we do only freezes the `encoder` and leaves the latent layers (i.e. VAE or linear encoding bottlenedck) and the `decoder` in a parameter group with the batchnorm layers.



#### parameter `Splitters`

> WARNING:  there are two completely different `splitter`s in the FastAI API.  This `splitter` groups the model parameters into groups for `freezing` and for progressive learning rates. (The other one is splits data into train and validate.  I got imminiently confused when I first started with the API by this.

> TODO:  more sophisticated parameter splitting to enable progressive learning rates

In [ ]:
#export

def bn_splitter(m):
    "splits all the batchnorm layers out"
    def _bn_splitter(l, g1, g2):
        if isinstance(l, nn.BatchNorm2d): g2 += l.parameters()
        elif hasattr(l, 'weight'): g1 += l.parameters()
        for ll in l.children(): _bn_splitter(ll, g1, g2)
        
    g1,g2 = [],[]
    _bn_splitter(m[0], g1, g2)
    
    g2 += m[1:].parameters()
    return g1,g2

def resnetVAE_split(m): 
    "simple splitter to freeze the non batch norm pre-trained encoder"
    to_freeze, dont_freeze = bn_splitter(m.encoder)
    #return L(to_freeze, dont_freeze + params(m.bn)+params(m.dec[:2]), params(m.dec[2:]))
    return L(to_freeze, dont_freeze + params(m.bn)+params(m.decoder))
    #return L(fz, nofz + params(m.bn)+params(m.dec[:6]), params(m.dec[6:]))
    

    
def AE_split(m): 
    "generic splitter for my AE classes- BVAE & AE & MMDVAE."
    to_freeze, dont_freeze = bn_splitter(m.encoder)
    return L(to_freeze, dont_freeze + params(m.bn)+params(m.decoder))
    

    


### other encoder and decoder types

> 1. MobileNet_v2 as the encoder, as a continuation of the original Sneaker Finder

> 2. simple bowtie convolutional encoder / decoder (Mimics the GOAT medium blog)
    - Architecture Hyperparameters:
        - Latent Size (research default 256, production default 32)
        - Filter Factor Size (research default 16, production default 32)
        - Latent Linear Hidden Layer Size (research default 2048, production default 1024)
        - The encoder architecture is as follows with research defaults from above:
            - Input 3x128x128 (conv2d block [conv2d, batchnorm2d, relu])
            - 16x64x64 (conv2d block [conv2d, batchnorm2d, relu])
            - 32x32x32 (conv2d block [conv2d, batchnorm2d, relu])
            - 64x16x16 (conv2d block [conv2d, batchnorm2d, relu])
            - 128x8x8 (conv2d block [conv2d, batchnorm2d, relu])
            - Flatten to 8192
            - 2048 (linear block [linear, batchnorm1d, relu])
            - Split the 2048 dimension into mu and log variance for the parameters of the latent distribution 
            - Latent mu size 256 (linear layer only with bias)
            - Latent logvar size 256 (linear layer only with bias)
        - In the middle here you can break out the BCE and KLD loss for the final loss term and use the standard reparam trick to sample from the latent distribution.
        - Decoder architecture an exact mirror 
            - Input 256
            - 2048 (linear block [linear, relu])
            - 8192 (linear block [linear, batchnorm1d, relu])
            - reshape (128x8x8)
            - 64x16x16 (conv2d transpose block [convtranspose2d, batchnorm2d, relu])
            - 32x32x32 (conv2d transpose block [convtranspose2d, batchnorm2d, relu])
            - 16x64x64 (conv2d transpose block [convtranspose2d, batchnorm2d, relu])
            - 3x128x128 (conv2d transpose [convtranspose2d, sigmoid]
        - For weight initialization I used a normal distribution centered at zero with 0.02 set for the stddev. Optimizer: Adam with default parameters, if I were to do it over again I'd spend more time here understanding the learning dynamics. The dataset was about ~10,000 with a 70/20/10 split, batch size 64, over 120 epochs, with a learning schedule to reduce when the loss started to plateau. No crazy image augmentation just resizing and standards flips. I used the ANN package Annoy to do the NN search for prod, normalizing the embeddings and using the cosine similarity, ANN factor was 128 for num_trees. 


> 3. MMD regularized VAE where the latents are drawn from a 

> TODO: Ranger optimizer might really help .. test

We can also use the transfer learning VAE tooling we previously built.  We just need to create the convolutional encoder and pass it in... Note that we don't have a pre-trained option, so DON'T FREEZE!

Now just wrap that simple conv block architecture into a _builder_.   And a meta-wrapper to let us call the conv_encoder and pre-trained options with the same function.   (I'll also put the `get_pretrained_parts` function here now even though we won't use it till the next section, so that we can make the `get_encoder_parts` generic wrapper handle both properly.)

In [ ]:
#export

#### TODO: refactor this to take a "BLOCK" input so we can have either ConvLayer or ResBlock pieces     
def get_conv_parts(im_size=IMG_SIZE):
    """
    make a simple convolutional ladder encoder
    """
    n_blocks = 5
    BASE = im_size//2**5        
    nfs = [3]+[(2**i)*BASE for i in range(n_blocks)] 
    n = len(nfs)

    modules =  [ConvLayer(nfs[i],nfs[i+1],
                            ks=5,stride=2,padding=2) for i in range(n - 1)]       

    return modules,nfs[-1],'vanilla'



def get_pretrained_parts(arch=resnet18):
    "this works for mobilnet_v2, resnet, and xresnet"
    cut = model_meta[arch]['cut']
    name = arch.__name__
    arch = arch(pretrained=True)
    enc_arch = list(arch.children())[:cut]
    enc_feats = 512
    return enc_arch, enc_feats, name



def get_encoder_parts(enc_type='vanilla',im_size=IMG_SIZE):
    encoder_parts = get_conv_parts(im_size=im_size) if isinstance(enc_type,str) else get_pretrained_parts(arch=enc_type)
    return encoder_parts # returns enc_arch,enc_dim,arch.__name__



In [ ]:
### TODO: Replace these training epochs with tests and maybe copy in some examples of the result?
##   


latent_dim = 128

# equalize KLDiv wrt errors per pixel
alpha = 3*IMG_SIZE*IMG_SIZE/latent_dim
alpha /= 20  # 5% retularizer

batchmean = True 
useL1 = False
hidden_dim = None
# SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True), 
cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),  ParamScheduler({'kl_weight': SchedNo(1.,1.) })]

metrics = default_AE_metrics(alpha,batchmean,useL1)

block = get_ae_DataBlock(aug=True)
batch_size = 64
dls = block.dataloaders(df, batch_size=batch_size)

arch='vanilla'

vae = AE(get_encoder_parts(arch), hidden_dim=hidden_dim,latent_dim=latent_dim, im_size=IMG_SIZE,out_range=OUT_RANGE)

# let beta be calculated by : 3*im_size*im_size/latent_dim
loss_func = AELoss(batchmean=batchmean,alpha=alpha,useL1=useL1)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split) #.to_fp16() #wd=config['wd'],opt_func=ranger,


> NOTE: The `to_fp16()` callbacks work but increasing the batch size doesn't really speed things up.


In [ ]:
latent_dim = 128

# equalize KLDiv wrt errors per pixel
alpha = 3*IMG_SIZE*IMG_SIZE/latent_dim
alpha /= 20  # 5% retularizer

batchmean = True 
useL1 = False
hidden_dim = None
# SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True), 
cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),  ParamScheduler({'kl_weight': SchedNo(1.,1.) })]

metrics = default_AE_metrics(alpha,batchmean,useL1)

block = get_ae_DataBlock(aug=True)
batch_size = 64
dls = block.dataloaders(df, batch_size=batch_size)

arch='vanilla'

vae = AE(get_encoder_parts(arch), hidden_dim=hidden_dim,latent_dim=latent_dim, im_size=IMG_SIZE,out_range=OUT_RANGE)

# let beta be calculated by : 3*im_size*im_size/latent_dim
loss_func = AELoss(batchmean=batchmean,alpha=alpha,useL1=useL1)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split) #.to_fp16() #wd=config['wd'],opt_func=ranger,
learn = learn.to_fp16()

### constructing VAE with Module Class Layers

For several of the decoder and "sampler" layers I might want to turn off the nonlinearity to give us more reasonable "gaussian" outputs to the Variational layer and the generated image which will is compared with the ImageNetStats batch-normalized image.



> IMPORTANT VAE TIP!!!   Make sure NOT to use batch normalization and non-linearity in the linear layers of the VAE.  The normalization will affect the representation and the KLD constraints.

In [ ]:
#export

class VAELinear(Module):
    "maps hidden (input) features to two latents (mu and logvar)"
    def __init__(self,in_features,latent_features):
        self.mu_linear = nn.Linear(in_features,latent_features)
        self.logvar_linear = nn.Linear(in_features,latent_features)

    def forward(self,h):
        #h = self.fc_in(h)
        return self.mu_linear(h), self.logvar_linear(h)

    
    
class VAELayer(Module):
    """
    The VAE : in_features to latent_features through 
        the "Variational" magic: "reparamaterization trick"
    """
    def __init__(self,in_features,latent_features):
        self.mu_logvar = VAELinear(in_features,latent_features)

    # 
    def reparam(self,mu,logvar):
        # should we pass through a deterministic code when not training?
        if False: return mu # self.training
    
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    
    def forward(self,h):
        mu,logvar = self.mu_logvar(h)
        #logvar = F.softplus(logvar)   # force logvar>0
        z = self.reparam(mu,logvar) # adds the noise by the reparam trick
        
        return z, mu, logvar
    
              

### simple BVAE class from VAE layer class components

This creates a pair of latents from which we can perform the "resample" trick. 

> NOTE:  this is a $\beta$-VAE (hence `BVAE` because we have a weighting factor for the KL Divergence regularazation factor which acts as a Legrangian).


Putting it all together gives us our VAE!   Note that we'll pass in the "parts" of the encoder for ease of using pretrained (or not) architectures.   The model name will correspond to the architecture of the encoder via `name`.

Note that the `BVAE` can simply inherit from the `AE` class we defined above.  Really the only difference in the `__init__` function is that a `VAELayer` which performs the reparameterization trick replaces the `AElayer` as `self.bn` 

In [ ]:
#export 
### TODO:  refactor the BVAE and AE to a single architecture... with a "sample" function ot 

class BVAE(AE):
    """
    simple VAE made with an encoder passed in, and some builder function for the Latent (VAE reparam trick) and decoder
    """
    def __init__(self,enc_parts,hidden_dim=None, latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE):
       
        """
        inputs:  
            enc_arch (pre-cut / pretrained)
            enc_dim
            latent_dim
            hidden_dim
            im_size,out_range
        """
        enc_arch,enc_feats,name = enc_parts

        # encoder
        #  arch,cut = xresnet18(pretrained=True),-4
        #  enc_arch = list(arch.children())[:cut]
        
        BASE = im_size//2**5
        enc_dim = enc_feats * BASE**2  # 2**(3*3) * (im_size//32)**2 #(output of resneet) #12800

        self.encoder = build_AE_encoder(enc_arch,enc_dim=enc_dim, hidden_dim=hidden_dim, im_size=im_size)

        in_dim = enc_dim if hidden_dim is None else hidden_dim

        # VAE Bottleneck
        self.bn = VAELayer(in_dim,latent_dim)     

        #decoder
        self.decoder = build_AE_decoder(hidden_dim=hidden_dim, latent_dim=latent_dim, im_size=im_size,out_range=out_range)

        store_attr('name,enc_dim, in_dim,hidden_dim,latent_dim,im_size,out_range') # do i need all these?


#     def decode(self, z):    
#         return self.decoder(z)

#     def encode(self, x):
#         h = self.encoder(x)
#         z, mu, logvar = self.bn(h) # reparam happens in the VAE layer
#         return z, mu, logvar

#     def forward(self, x):
#         #z, mu, logvar = self.encode(x)
#         #         h = self.encoder(x)
#         #         z, mu, logvar = self.bn(h) # reparam happens in the VAE layer
#         #         x_hat = self.decoder(z)
        
#         z,mu,logvar = self.encode(x)
#         x_hat = self.decode(z)
#         latents = torch.stack([mu,logvar],dim=-1)
        
#         return x_hat, latents # assume dims are [batch,latent_dim,concat_dim]

    
# # AE 
#    def decode(self, z):    
#         return self.decoder(z)
    
#     def encode(self, x):
#         h = self.encoder(x)
#         return self.bn(h)
            
#     def forward(self, x):
#         """
#         pass the "latents" out to keep the learn mechanics consistent... 
#         """
#         h = self.encoder(x)
#         z,logvar = self.bn(h)
#         x_hat = self.decoder(z)                    
#         latents = torch.stack([z,logvar] ,dim=-1)

#         return x_hat , latents
    


We defined a nice wrapper for building the encoder parts above.

```python
def get_pretrained_parts(arch=resnet18):
    "this works for mobilnet_v2, resnet, and xresnet"
    cut = model_meta[arch]['cut']
    name = arch.__name__
    arch = arch(pretrained=True)
    enc_arch = list(arch.children())[:cut]
    enc_feats = 512
    return enc_arch, enc_feats, name
```

Sweet, we've verified the arcitecture works, but we need to train it with a loss that constrains the variational layers with the KL Divergence.  Otherwise the simple MSE will diverge.  

### VAE Loss functions classes

This simply adds a KLD regularizer to the latent space defined by two rank-1 tensors defining gaussian-prior latents.  i.e. a mean ($\mu$) and standard deviation ($\sigma$).  

> NOTE: for convenience and numeric stability the $\sigma$ is representated as a $\log(\sigma^{s})$ so the tensors are called `mu` and `logvar`


In [ ]:
#export

# called `after_batch`

class BVAELoss(Module):
    """
    Measures how well we have created the original image, 
    plus the KL Divergence with the unit normal distribution
    batchmean option sums first and averages over batches (for smaller total error magnitudes.. cosmentic)
    """
    def __init__(self, batchmean=False, alpha=1.0,useL1=False):
        """
        reduction 'sum', else 'batchmean'
        
        """
        pix_loss = MSELossFlat(reduction='sum') if not useL1 else L1LossFlat(reduction='sum')
        _KLD = KLDiv(batchmean=False) # force to full sum
        store_attr('pix_loss,alpha,batchmean,_KLD')
        
        
    def forward(self, preds, *target):
        """
        pred =(x_hat,KLD,kl_weight) #mu,log_var, kl_weight)
        target is x (original) 
        """
        
        # this handles the annealed kl_weight and passing the mu,logvar around we added...
        if(len(preds) == 3):
            x_hat, latents, kl_weight = preds
        else: #if len(preds) == 2:  # we should never get here... unless we delete the callback
            x_hat, latents = preds
            
            kl_weight = x_hat[0].new(1)
            kl_weight[0] = 1.0
        
        mu, logvar = latents.split(1,dim=2)

        #note: both mse and KLD are summing errors over batches, and pixels or latents 
        pix_err = self.pix_loss(x_hat, target[0]) 
        kld_err = self.alpha * self._KLD(mu,logvar).sum() #_KLD doesn't sum over batches by default
        total =  (pix_err + kld_err*kl_weight) 
        if self.batchmean: total *= (1./mu.size()[0])
        return total

    


In [ ]:
# export

def default_VAE_metrics(alpha,batchmean,useL1):
    "long default list of metrics for the VAE"
    
    first = L2BMeanMetric() if batchmean else L2Metric()
    second = L1BMeanMetric() if batchmean else L1Metric()
    
    if useL1: first,second = second,first
        
    metrics = [first,
                MuMetric(), 
                StdMetric(),
                LogvarMetric(),
                WeightedKLDMetric(batchmean=batchmean,alpha=alpha),
                KLWeightMetric(), 
                second,
                MuSDMetric(), 
                StdSDMetric(), 
                LogvarSDMetric(),              
               ]
    return metrics



def short_VAE_metrics(alpha,batchmean,useL1):
    "short default list of metrics for the AE"
    
    first = L2BMeanMetric() if batchmean else L2MeanMetric()
    second = L1BMeanMetric() if batchmean else L2MeanMetric()
    
    if useL1: first,second = second,first
        
    metrics = [first,
                MuMetric(), 
                StdMetric(),
                LogvarMetric(),
                WeightedKLDMetric(batchmean=batchmean,alpha=alpha)              
               ]
    return metrics




Here's how we put everything together.  

In [ ]:
latent_dim = 128

alpha = 5
batchmean = True
useL1 = False
hidden_dim = None


cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(), ParamScheduler({'kl_weight':  SchedNo(1.,1.) })]

metrics = default_VAE_metrics(alpha,batchmean,useL1)



We can use this callback if we want to save the model at every epoch.  Which could be super useful if we were able to actually overfit our model. 
> 'SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True)' 




In [ ]:
#hide
block = get_ae_DataBlock(aug=True)

batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)

enc_parts = get_pretrained_parts(arch=resnet18)

rnet_vae = BVAE(enc_parts, hidden_dim=hidden_dim,latent_dim=latent_dim, im_size=IMG_SIZE,out_range=OUT_RANGE)
    
loss_func = BVAELoss(batchmean=batchmean,alpha=alpha,useL1=False)

learn = Learner(dls, rnet_vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split)

learn.freeze()

In [ ]:

# NOTE: lf_finder does NOT work correctly with our annealed kl_weight... 

lr1,lr2=learn.lr_find()
mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


> NOTE: The CallBacks need to be updated for the KL loss annealing schedule as we tune.  We want to turn the `kl_weight` to 1.0 for instance when using the learning rate finder `learn.lr_find()`, and after an initial _burn in_ where the KL_loss term gradually ramps in, setting the `kl_weight` to stay at unity will be useful as we separately turn the learning rate (e.g. `fit_one_cycle` or `fit_flat_cosine`.


In [ ]:

#put in the annealied KL_weight...
learn.remove_cb(learn.cbs[-1])
# add new constant scheduler

learn.add_cb(ParamScheduler({'kl_weight': default_KL_anneal_in()} ) )



In [ ]:

n_epochs = 5

#learn.fit_one_cycle(n_epochs,lr_max= lr1)
learn.fit_one_cycle(n_epochs)
learn.show_results()


The vae with pretrained resnet encoder seems to train to a much better end-point if we keep the resnet frozen.  Hence the commented out `learn.unfreeze()` below.

In [ ]:
# replace kl_weight scheduler... with a constant unity... (NOTE: should be robust to failure if we don't replace it)
learn.remove_cb(learn.cbs[-1])
# add new constant scheduler
learn.add_cb(ParamScheduler({'kl_weight':  SchedNo(1.,1.) }) )

#learn.unfreeze()


In [ ]:

epochs = 20

#learn.fit_one_cycle(epochs, lr_max= 1e-3)
learn.fit_flat_cos(epochs,pct_start=.05)

In [ ]:
learn.show_results()

### "_Vanilla_" convolutional beta-VAE

In [ ]:
latent_dim = 128

# equalize KLDiv wrt errors per pixel
#  alpha = 3*IMG_SIZE*IMG_SIZE/latent_dim
alpha = 5
batchmean = True
useL1 = False

# SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True), 
cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(), ParamScheduler({'kl_weight': default_KL_anneal_in() })]

metrics = default_VAE_metrics(alpha,batchmean,useL1)


block = get_ae_DataBlock(aug=True)
batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)

arch='vanilla'
vae = BVAE(get_encoder_parts(arch), hidden_dim=None,latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE)
                   
# let beta be calculated by : 3*im_size*im_size/latent_dim
loss_func = BVAELoss(batchmean=batchmean,alpha=alpha,useL1=False)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split)#.to_fp16() #wd=config['wd'],opt_func=ranger,






##  MMD-VAE
The MMD replace latent regularization term in loss_fn (KLD) with Maximal Meanm Discrepancy.

We'll make an MMDVAE class to keep things declarative, but its really just an AE. i.e. a linear latent layer.

Additional background on MME from [https://github.com/Saswatm123/MMD-VAE]:

> Maximum Mean Discrepancy Variational Autoencoder, a member of the InfoVAE family that maximizes Mutual Information between the Isotropic Gaussian Prior (as the latent space) and the Data Distribution.\
Short explanation: The traditional VAE is known as the ELBO-VAE, named after the Evidence Lower Bound used in its objective. The ELBO suffers from two problems: overestimation of latent variance, and uninformative latent information.\
The latter is because one of the objective's terms is the KL-Divergence between the Gaussian parameterized by the encoder and the Standard Isotropic Gaussian. This dissuades usage of the latent code, so that the KL-Divergence term is allowed to fall even further. It is important to note that the KL-Divergence should never truly reach zero, as that means the encoder is not learning useful features and cannot find feature locality, and the decoder is just randomly sampling from Standard Gaussian noise.\
The overestimation of variance results from the KL-Divergence term not being strong enough to balance against the Reconstruction Error, and thus the Encoder prefers to learn a multimodal latent distribution with spread apart means, leading to low training error as it overfits, but low quality samples as well, as the sampling distribution is assumed to be a Standard Isotropic Gaussian. One effort to mitigate this effect is the Disentangled Variational Autoencoder, which simply raises the weight on the KL-Divergence term. However, this increases the problem stated in the paragraph above since it further penalizes using the latent code.\
For more detailed explanations, I used these resources to learn, in order of usefulness to me:\
    - https://arxiv.org/pdf/1706.02262.pdf \
    - http://ruishu.io/2018/03/14/vae/ \
    - http://approximateinference.org/accepted/HoffmanJohnson2016.pdf \
    - https://ermongroup.github.io/blog/a-tutorial-on-mmd-variational-autoencoders/ \
    - http://bjlkeng.github.io/posts/variational-bayes-and-the-mean-field-approximation/ \
    - https://ermongroup.github.io/cs228-notes/inference/variational/ \

In [ ]:
# export

def gaussian_kernel(a, b):
    "helper for computing MMD"
    dim1_1, dim1_2 = a.shape[0], b.shape[0]
    depth = a.shape[1]
    a = a.view(dim1_1, 1, depth)
    b = b.view(1, dim1_2, depth)
    a_core = a.expand(dim1_1, dim1_2, depth)
    b_core = b.expand(dim1_1, dim1_2, depth)
    numerator = (a_core - b_core).pow(2).mean(2)/depth
    return torch.exp(-numerator)

def MMD(a, b):
    "Max Mean Discrepancy"
    return gaussian_kernel(a, a).mean() + gaussian_kernel(b, b).mean() - 2*gaussian_kernel(a, b).mean()
        
def rawMMD(a, b):
    "_raw_ values from gauss kernals, assuming that and b have the same shape"
    return gaussian_kernel(a, a) + gaussian_kernel(b, b) - 2*gaussian_kernel(a, b)



# the MMDVAE is built on the basic AE archiecure
class MMDVAE(AE): pass
    

class MaxMeanDiscrepancy(Module):
    """
     MMD
     add alpha?
    """
    def __init__(self, batchmean=False):
        """
        reduction 'mean', else 'batchmean' means only over batch
        
        """
        MMD = self._MMDsum if batchmean else self._MMDmean
        store_attr('batchmean,MMD')
    
    def _gaus_ker(self,a, b):
        "gaussian kernal"
        dim1_1, dim1_2 = a.shape[0], b.shape[0]
        depth = a.shape[1]
        numerator = 1.0/depth
        a = a.view(dim1_1, 1, depth)
        b = b.view(1, dim1_2, depth)
        a_core = a.expand(dim1_1, dim1_2, depth)
        b_core = b.expand(dim1_1, dim1_2, depth)
        a_m_b = a_core - b_core
        numerator *= (a_m_b*a_m_b).mean(2)  
        #numerator = (a_core - b_core).pow(2).mean(2)   /depth
        return torch.exp(-numerator)
    
    def _rawMMD(self, a, b):
        return self._gaus_ker(a, a) +  self._gaus_ker(b, b) - 2*self._gaus_ker(a, b)
    
    def _MMDmean(self, a, b):
        return self._rawMMD( a, b).mean()

    def _MMDsum(self, a, b):
        return self._rawMMD( a, b).sum()
    

    def forward(self,true_samples, latent):
#         bs = latents.size()[0]
#         latent_dim = z.size()[1]
#         true_samples = torch.randn((bs,latent_dim), requires_grad=False).cuda()
        mmd = self.MMD(true_samples, latent)        
        return mmd
        
        
class MMDLoss(Module):
    """
    Measures mean square error of prediction and original image, 
    regularized by MMD.
    
    Note: using reuction = 'mean' because it keeps the regularization relatively potent (i.e. pixels>>latents)
    """
    def __init__(self, batchmean=False, alpha=1.0,useL1=False):
        """
        reduction 'sum', else 'batchmean'
        
        """
        if batchmean:
            pix_loss = MSELossFlat(reduction='sum') if not useL1 else L1LossFlat(reduction='sum') 
            #mmd = _MMDsum
        else:
            pix_loss = MSELossFlat(reduction='mean') if not useL1 else L1LossFlat(reduction='mean') 
            #mmd = _MMD

        mmd = MaxMeanDiscrepancy(batchmean=batchmean)
        store_attr('pix_loss,alpha,batchmean,mmd')
        
        
    def forward(self, preds, *target):
        """
        pred =(x_hat,KLD,kl_weight) #mu,log_var, kl_weight)
        target is x (original) 
        """
        
        # this handles the annealed kl_weight and passing the mu,logvar around we added...
        if(len(preds) == 3):
            x_hat, latents, kl_weight = preds
        else: #if len(preds) == 2:  # we should never get here... unless we delete teh callback
            x_hat, latents = preds
            
            kl_weight = x_hat[0].new(1)
            kl_weight[0] = 1.0
        
        z, _ = latents.split(1,dim=2)

        #note: both mse and KLD are summing errors over batches, and pixels or latents 
        pix_err = self.pix_loss(x_hat, target[0]) 
        

        bs = latents.size()[0]
        latent_dim = z.size()[1]
        true_samples = torch.randn((bs,latent_dim), requires_grad=False).cuda()
        mmd_loss = self.mmd(true_samples, z) * self.alpha    
    
        total =  (pix_err + mmd_loss*kl_weight) 
        total *= (1./bs) if self.batchmean else 1.0
        return total

            

    
class MMDMetric(MyMetric):
    def __init__(self,batchmean=False,alpha=1.0): 
        vals = []
        #mmd = _MMDsum if batchmean else _MMD
        mmd = MaxMeanDiscrepancy(batchmean=batchmean)

        store_attr('vals,batchmean,alpha,mmd')
        
    def accumulate(self, learn):        
        latents = learn.pred[1]
        z, _ = latents.split(1,dim=2)

        bs = latents.size()[0]
        latent_dim = z.size()[1]
        true_samples = torch.randn((bs,latent_dim), requires_grad=False).cuda()
        mmd_loss = self.mmd(true_samples, z)   
        mmd_loss *= (self.alpha/bs) if self.batchmean else self.alpha

        self.vals.append(to_detach(mmd_loss))

# export

def short_MMEVAE_metrics(alpha,batchmean,useL1):
    "short list of metrics for the VAE"
    
    first = L2BMeanMetric() if batchmean else L2MeanMetric()
    second = L1BMeanMetric() if batchmean else L1MeanMetric()
    
    if useL1: first,second = second,first
        
    metrics = [first,
                MMDMetric(batchmean=batchmean,alpha=alpha), 
                MuMetric(), 
                MuSDMetric(), 
                ]
    return metrics

def default_MMEVAE_metrics(alpha,batchmean,useL1):
    "long default list of metrics for the VAE"
    
    first = L2BMeanMetric() if batchmean else L2MeanMetric()
    second = L1BMeanMetric() if batchmean else L1MeanMetric()
    
    if useL1: first,second = second,first
        
    metrics = [first,
                MMDMetric(batchmean=batchmean,alpha=alpha), 
                MuMetric(),
                StdMetric(),
                second,
                MuSDMetric(), 
                LogvarMetric(),
                L1LatentReg(batchmean=batchmean,alpha=alpha),
                WeightedKLDMetric(batchmean=batchmean,alpha=alpha),
                LogvarSDMetric()]               
    return metrics




#### Vanilla MMD VAE

Simply call our `get_encoder_parts` with `arch='vanilla'` in the `MMDVAE` builder. 

In [ ]:
latent_dim = 128

cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),         
               ParamScheduler({'kl_weight': SchedNo(1.,1.) })]
alpha = 20

batchmean = True
useL1 = False
hidden_dim = None
metrics = default_MMEVAE_metrics(alpha,batchmean,useL1)


batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)

arch='vanilla'
vae = MMDVAE(get_encoder_parts(arch), hidden_dim=hidden_dim,latent_dim=latent_dim, im_size=IMG_SIZE,out_range=OUT_RANGE)

loss_func = MMDLoss(batchmean=batchmean,alpha=alpha,useL1=useL1)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split) #.to_fp16() #wd=config['wd'],opt_func=ranger,
    

### ResNet Encoder MMDVAE

We build these by passing `resnet18` (_not_ `'resnet18'`) to `get_encoder_parts` helper for the parts to init the `MMDVAE`.

First a traditional "fine_tune" type of training: 

In [ ]:

latent_dim = 128

cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),         
               ParamScheduler({'kl_weight': SchedNo(1.,1.) })]

alpha = 10

batchmean = True
useL1 = False
hidden_dim = None
metrics = default_MMEVAE_metrics(alpha,batchmean,useL1)


batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)


arch=resnet18
vae = MMDVAE(get_encoder_parts(arch), hidden_dim=hidden_dim,latent_dim=latent_dim, im_size=IMG_SIZE,out_range=OUT_RANGE)

loss_func = MMDLoss(batchmean=batchmean,alpha=alpha,useL1=useL1)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split) #.to_fp16() #wd=config['wd'],opt_func=ranger,
    
learn.freeze()


In [ ]:
# warm things up...  low-ish learning rate + anealing kldiv retularizaiton into
learn.freeze()
n_epoch = 20
#learn.fit_flat_cos(n_epoch) #, lr=1e-3, div_final=1e6, pct_start=0.2)
learn.fit_flat_cos(n_epoch)#, lr=lr1, div_final=1e5, pct_start=0.5)
learn.fit_one_cycle(n_epoch)#,lr_max=gmlr) #, lr_max= base_lr)

learn.show_results()

In [ ]:
# warm things up...  low-ish learning rate + anealing kldiv retularizaiton into
n_epoch = 20
learn.unfreeze()
#learn.fit_flat_cos(n_epoch, lr=lr1, div_final=1e6, pct_start=0.7)
#learn.fit_flat_cos(n_epoch, lr=1e-3, div_final=1e5, pct_start=0.5)
learn.fit_one_cycle(n_epoch) #, lr_max= base_lr)

learn.show_results()


#### vanilla-resnet MMD VAE
##### no freeze

We build these by passing `resnet18` (_not_ `'resnet18'`) to `get_encoder_parts` helper for the parts to init the `MMDVAE`.

> NOTE: our architecture trains best when simply starting with the pretrained weights.  Trying to "fine_tune" by training the backend on a _frozen_ resnet and then unfreezing doesn't work with the parameter groupings from the `AE_split`.  The `learn.unfreeze()` doesn't actually do anythign (_unfrozen_ is default) but makes is clear we are _un_ -frozen

In [ ]:

latent_dim = 128

cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),         
               ParamScheduler({'kl_weight': SchedNo(1.,1.) })]


alpha = 10

batchmean = True
useL1 = False
hidden_dim = None
metrics = default_MMEVAE_metrics(alpha,batchmean,useL1)


batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)


arch=resnet18
vae = MMDVAE(get_encoder_parts(arch), hidden_dim=hidden_dim,latent_dim=latent_dim, im_size=IMG_SIZE,out_range=OUT_RANGE)

loss_func = MMDLoss(batchmean=batchmean,alpha=alpha,useL1=useL1)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split) #.to_fp16() #wd=config['wd'],opt_func=ranger,
    
    
    
learn.unfreeze()


## vanilla-resnet MMD-VAE
##### ResBlocks replacing Conv2d in encoder AND decoder.
'Vanilla' encoder architecture made with `ResBlock` layers instead of `ConvLayer` layers and `Mish` activation.

In [ ]:
# hide
## This block is for experimenting with making a builder which can take ResBlock or ConvLayers (ConvBlocks?  ConvLayer(ni,nf),ConvLayer(nf,nf)??

def get_conv_parts2(enc_type='vanilla',im_size=IMG_SIZE):
    """
    make a simple convolutional ladder encoder
    TODO:  make a switch on enc_type
        - vanilla, resnet or vanilla-res
        -TODO: change to 'convblock' and 'resblock'
    """
    n_blocks = 5
    BASE = im_size//2**5        
    nfs = [3]+[(2**i)*BASE for i in range(n_blocks)] 
    n = len(nfs)

    
    
    modules =  [ConvLayer(nfs[i],nfs[i+1],
                            ks=5,stride=2,padding=2) for i in range(n - 1)]       
    
#     modules =  [ResBlock(1, nfs[i],nfs[i+1], 
#                          stride=2, act_cls=Mish)  for i in range(n - 1)]       

    return modules,nfs[-1],'vanilla'


def get_encoder_parts2(enc_type='vanilla',im_size=IMG_SIZE):
    encoder_parts = get_conv_parts2(enc_type=enc_type,im_size=im_size) if isinstance(enc_type,str) else get_pretrained_parts(arch=enc_type)
    return encoder_parts # returns enc_arch,enc_dim,arch.__name__


In [ ]:
#export
class UpsampleResBlock(Module):
    def __init__(self, up_in_c:int, final_div:bool=True, blur:bool=False, **kwargs):
        """
        Upsampling using PixelShuffle_INCR and ResBlocks
        - up_in_c :  "Upsample input channel"
        """
        self.shuf = PixelShuffle_ICNR(up_in_c, up_in_c//2, blur=blur, **kwargs)
        ni = up_in_c//2
        nf = ni if final_div else ni//2
        self.conv1 = ResBlock(1,ni, nf, **kwargs) # since we'll apply it by hand...
        self.conv2 = ResBlock(1,nf, nf, **kwargs)

    def forward(self, up_in:Tensor) -> Tensor:
        up_out = self.shuf(up_in)
        return self.conv2(self.conv1(up_out))
    


def get_resblockencoder_parts(enc_type='vanilla',im_size=IMG_SIZE):
    """
    make a simple (hence 'vanilla') convolutional ladder encoder with ResBlock parts
    """
    n_blocks = 5
    BASE = im_size//2**5        
    nfs = [3]+[(2**i)*BASE for i in range(n_blocks)] 
    n = len(nfs)

    
    modules =  [ResBlock(1, nfs[i],nfs[i+1], 
                          stride=2, act_cls=Mish)  for i in range(n - 1)]       

    return modules,nfs[-1],'resblock'



# def build_ResBlockAE_decoder(hidden_dim=2048, latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE):
#         BASE = im_size//2**5
#         #store_attr('enc_dim,latent_dim, hidden_dim,im_size')

        

#         #decoder
#         n_blocks = 5        
#         nfs = [3] + [2**i*n_blocks for i in range(n_blocks+1)] 
#         nfs.reverse()
#         n = len(nfs)
        
#         modules =  [UpsampleResBlock(nfs[i]) for i in range(n - 2)]        
#         decoder = nn.Sequential( LinBnDrop(latent_dim,hidden_dim,
#                                                 bn=True,# batch normalizaiton shouldn't be a problem here
#                                                 p=0.0,act=nn.ReLU(),lin_first=True),
#                                      LinBnDrop(hidden_dim,im_size*n_blocks*n_blocks,
#                                                 bn=True,# batch normalizaiton shouldn't be a problem here
#                                                 p=0.0,act=nn.ReLU(),lin_first=True),
#                                       ResizeBatch(im_size,n_blocks,n_blocks),
#                                       *modules,
#                                       ResBlock(1,nfs[-2],nfs[-1],
#                                                 ks=1,padding=0, norm_type=None, #act_cls=nn.Sigmoid) )
#                                                 act_cls=partial(SigmoidRange, *out_range)))
        
#         return decoder


class ResBlockAEDecoder(Module):
    def __init__(self, hidden_dim=None, latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE):
        """
        Decoder Module made of ResBlocks returning the latent representation back into an "image"
            latent_dim - dimension of latent representation
            hidden_dim - optional additional linear layer between the latent and decoder
            im_size - passed to make sure we are scaling back to the right size
            out_range - ensures the output is on teh same scale as the _normalized_ input image
        """
        #decoder
        n_blocks = 5 
        BASE = im_size//2**5               
        
        hidden = im_size*BASE*BASE if hidden_dim is None else hidden_dim
        z_fc = [nn.Linear(latent_dim,hidden)]
        if hidden_dim:  # i.e. is not None
            z_fc += [nn.Linear(hidden,im_size*BASE*BASE)]
        
        nfs = [3] + [2**i*BASE for i in range(n_blocks+1)] 
        nfs.reverse()
        n = len(nfs)
        
        modules =  [UpsampleResBlock(nfs[i]) for i in range(n - 2)]        
        self.decoder = nn.Sequential(*z_fc,
                                      ResizeBatch(im_size,BASE,BASE),
                                      *modules,
                                      ResBlock(1,nfs[-2],nfs[-1],
                                                ks=1,padding=0, norm_type=None, #act_cls=nn.Sigmoid) )
                                                act_cls=partial(SigmoidRange, *out_range)))
        
        store_attr('latent_dim, hidden_dim,im_size,out_range')

    def forward(self, z):    
        z = self.decoder(z)
        return z
            
    
def build_ResBlockAE_decoder(hidden_dim=None, latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE):
    "wrapper to sequential-ize ResBlockAEDecoder class"
    decoder = ResBlockAEDecoder(hidden_dim=hidden_dim, latent_dim=latent_dim, im_size=im_size,out_range=out_range)
    return nn.Sequential(*list(decoder.children()))
    

    
class ResBlockAE(AE):
    def __init__(self,enc_parts,hidden_dim=None, latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE,isVAE=False):
       
        """
        inputs:  
            enc_parts - encoder architecture 
            latent_dim - dimension of latent representation
            hidden_dim - optional additional linear layer between the latent and decoder
            im_size - passed to make sure we are scaling back to the right size
            out_range - ensures the output is on teh same scale as the _normalized_ input image
            isVae - switch for the type of latent representation
            
        """
        enc_arch,enc_feats,name = enc_parts
        
        BASE = im_size//2**5
        enc_dim = enc_feats * BASE**2  # 2**(3*3) * (im_size//32)**2 #(output of resneet) #12800

        #encoder
        self.encoder = build_AE_encoder(enc_arch,enc_dim=enc_dim, hidden_dim=hidden_dim, im_size=im_size)

        in_dim = enc_dim if hidden_dim is None else hidden_dim
        
        # AE Bottleneck
        latent = VAELayer if isVAE else LatentLayer
        
        self.bn = latent(in_dim,latent_dim)     
        
        #decoder
        self.decoder = build_ResBlockAE_decoder(hidden_dim=hidden_dim, latent_dim=latent_dim, im_size=im_size,out_range=out_range)

        store_attr('name,enc_dim, in_dim,hidden_dim,latent_dim,im_size,out_range') # do i need all these?
        

    
        
#     def decode(self, z):    
#         return self.decoder(z)
    
#     def encode(self, x):
#         h = self.encoder(x)
#         return self.bn(h)
            
#     def forward(self, x):
#         """
#         pass the "latents" out to keep the learn mechanics consistent... 
#         """
#         h = self.encoder(x)
#         z,logvar = self.bn(h)
#         x_reconst = self.decoder(z)                    
#         latents = torch.stack([z,logvar] ,dim=-1)

#         return x_reconst , latents
    

Finally we can use the `ResBlockBVAE` which instantiates a `ResBlock` `decoder` to optimize the architecture.  This is following the fastAI API lessong from the "bag of tricks" ResNet paper (CITATION), which is a general true-ism which could be glibly states as: "replacing a `Conv` with a `ResBlock` always gets you better results".  The Class constructor `ResBlockAE` takes `isVAE` to switch between `AELayer` and `LatentLayer` latents.

> TODO:  update AE class to make a VAE or AE based on the `isVAE` switch

In [ ]:
latent_dim = 128
alpha = 5 # doubled because latent is half?
batchmean = True
useL1 = False

# SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True), 
cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(), ParamScheduler({'kl_weight':  SchedNo(1.,1.) })]


metrics = default_VAE_metrics(alpha,batchmean,useL1)


block = get_ae_DataBlock(aug=True)
batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)

arch='resnblock'
vae = ResBlockAE(get_resblockencoder_parts(arch), hidden_dim=2048,latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE,isVAE= True)
                   
# let beta be calculated by : 3*im_size*im_size/latent_dim
loss_func = BVAELoss(batchmean=batchmean,alpha=alpha,useL1=False)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split)#.to_fp16() #wd=config['wd'],opt_func=ranger,


## next steps.

- visualize activations in encoder decoder
- improve training with progressive learning rates
- G

## export / provenance

In [ ]:
#hide

from nbdev.export import notebook2script
notebook2script()


### 128 latents, alpha=10

In [ ]:
latent_dim = 128

# cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),         
#                SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True),
#                ParamScheduler({'kl_weight': SchedNo(1.,1.) })]
cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),         
               ParamScheduler({'kl_weight': SchedNo(1.,1.) })]

alpha = 10
# note that alpha needs to be adjusted to scale MMD regularizer compared to error for batchmean=true
#.  e.g.  *= 3*IMG_SIZE**2/latent_dim
batchmean = True
useL1 = False
hidden_dim = None

metrics = default_MMEVAE_metrics(alpha,batchmean,useL1)

batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)

arch='resblock'
vae = ResBlockAE(get_resblockencoder_parts(arch), hidden_dim=hidden_dim,latent_dim=latent_dim,  im_size=IMG_SIZE,out_range=OUT_RANGE)
  
# let beta be calculated by : 3*im_size*im_size/latent_dim
loss_func = MMDLoss(batchmean=batchmean,alpha=alpha,useL1=False)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split)#.to_fp16() #wd=config['wd'],opt_func=ranger,
     
    
    
    

    

In [ ]:
lr1,lr2=learn.lr_find()
mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
# warm things up...  low-ish learning rate + anealing kldiv retularizaiton into
#learn.freeze()
n_epoch = 200
#learn.fit_flat_cos(n_epoch) #, lr=1e-3, div_final=1e6, pct_start=0.2)
learn.fit_flat_cos(n_epoch)#, lr=lr1, div_final=1e5, pct_start=0.5)
#learn.fit_one_cycle(n_epoch,lr_max=gmlr) #, lr_max= base_lr)

learn.show_results()

In [ ]:
prefix = f"MMDVae-{'TMP'}-latent{latent_dim}"
filename = f"frozen-{prefix}-{learn.model.name}-alpha{alpha:d}_{datetime.now().strftime('%Y-%m-%d_%H.%M.%S')}"

learn.save(filename)
#learn.export(f'{filename}.pkl')

### 128 latents, alpha=20

In [ ]:
latent_dim = 128

# cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),         
#                SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True),
#                ParamScheduler({'kl_weight': SchedNo(1.,1.) })]
cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),         
               ParamScheduler({'kl_weight': SchedNo(1.,1.) })]

alpha = 20
# note that alpha needs to be adjusted to scale MMD regularizer compared to error for batchmean=true
#.  e.g.  *= 3*IMG_SIZE**2/latent_dim
batchmean = True
useL1 = False
hidden_dim = None

metrics = default_MMEVAE_metrics(alpha,batchmean,useL1)

batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)

arch='resblock'
vae = ResBlockAE(get_resblockencoder_parts(arch), hidden_dim=hidden_dim,latent_dim=latent_dim,  im_size=IMG_SIZE,out_range=OUT_RANGE)
  
# let beta be calculated by : 3*im_size*im_size/latent_dim
loss_func = MMDLoss(batchmean=batchmean,alpha=alpha,useL1=False)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split)#.to_fp16() #wd=config['wd'],opt_func=ranger,
     
    
    

In [ ]:
lr1,lr2=learn.lr_find()
mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
# warm things up...  low-ish learning rate + anealing kldiv retularizaiton into
#learn.freeze()
n_epoch = 200
#learn.fit_flat_cos(n_epoch) #, lr=1e-3, div_final=1e6, pct_start=0.2)
learn.fit_flat_cos(n_epoch)#, lr=lr1, div_final=1e5, pct_start=0.5)
#learn.fit_one_cycle(n_epoch,lr_max=gmlr) #, lr_max= base_lr)

learn.show_results()

In [ ]:
prefix = f"MMDVae-{'TMP'}-latent{latent_dim}"
filename = f"frozen-{prefix}-{learn.model.name}-alpha{alpha:d}_{datetime.now().strftime('%Y-%m-%d_%H.%M.%S')}"

learn.save(filename)
#learn.export(f'{filename}.pkl')

### 64 latents, alpha=10

In [ ]:
latent_dim = 64

# cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),         
#                SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True),
#                ParamScheduler({'kl_weight': SchedNo(1.,1.) })]
cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),         
               ParamScheduler({'kl_weight': SchedNo(1.,1.) })]

alpha = 10
# note that alpha needs to be adjusted to scale MMD regularizer compared to error for batchmean=true
#.  e.g.  *= 3*IMG_SIZE**2/latent_dim
batchmean = True
useL1 = False
hidden_dim = None

metrics = default_MMEVAE_metrics(alpha,batchmean,useL1)

batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)

arch='resblock'
vae = ResBlockAE(get_resblockencoder_parts(arch), hidden_dim=hidden_dim,latent_dim=latent_dim,  im_size=IMG_SIZE,out_range=OUT_RANGE)
  
# let beta be calculated by : 3*im_size*im_size/latent_dim
loss_func = MMDLoss(batchmean=batchmean,alpha=alpha,useL1=False)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split)#.to_fp16() #wd=config['wd'],opt_func=ranger,
     
    
    

In [ ]:
lr1,lr2=learn.lr_find()
mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
# warm things up...  low-ish learning rate + anealing kldiv retularizaiton into
#learn.freeze()
n_epoch = 200
#learn.fit_flat_cos(n_epoch) #, lr=1e-3, div_final=1e6, pct_start=0.2)
learn.fit_flat_cos(n_epoch)#, lr=lr1, div_final=1e5, pct_start=0.5)
#learn.fit_one_cycle(n_epoch,lr_max=gmlr) #, lr_max= base_lr)

learn.show_results()

In [ ]:
prefix = f"MMDVae-{'TMP'}-latent{latent_dim}"
filename = f"frozen-{prefix}-{learn.model.name}-alpha{alpha:d}_{datetime.now().strftime('%Y-%m-%d_%H.%M.%S')}"

learn.save(filename)
#learn.export(f'{filename}.pkl')

### 64 latents, alpha = 20

In [ ]:
latent_dim = 64

# cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),         
#                SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True),
#                ParamScheduler({'kl_weight': SchedNo(1.,1.) })]
cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(),         
               ParamScheduler({'kl_weight': SchedNo(1.,1.) })]

alpha = 20
# note that alpha needs to be adjusted to scale MMD regularizer compared to error for batchmean=true
#.  e.g.  *= 3*IMG_SIZE**2/latent_dim
batchmean = True
useL1 = False
hidden_dim = None

metrics = default_MMEVAE_metrics(alpha,batchmean,useL1)

batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)

arch='resblock'
vae = ResBlockAE(get_resblockencoder_parts(arch), hidden_dim=hidden_dim,latent_dim=latent_dim,  im_size=IMG_SIZE,out_range=OUT_RANGE)
  
# let beta be calculated by : 3*im_size*im_size/latent_dim
loss_func = MMDLoss(batchmean=batchmean,alpha=alpha,useL1=False)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split)#.to_fp16() #wd=config['wd'],opt_func=ranger,
     
    
    

In [ ]:
lr1,lr2=learn.lr_find()
mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
# warm things up...  low-ish learning rate + anealing kldiv retularizaiton into
#learn.freeze()
n_epoch = 200
#learn.fit_flat_cos(n_epoch) #, lr=1e-3, div_final=1e6, pct_start=0.2)
learn.fit_flat_cos(n_epoch)#, lr=lr1, div_final=1e5, pct_start=0.5)
#learn.fit_one_cycle(n_epoch,lr_max=gmlr) #, lr_max= base_lr)

learn.show_results()

In [ ]:
prefix = f"MMDVae-{'TMP'}-latent{latent_dim}"
filename = f"frozen-{prefix}-{learn.model.name}-alpha{alpha:d}_{datetime.now().strftime('%Y-%m-%d_%H.%M.%S')}"

learn.save(filename)
#learn.export(f'{filename}.pkl')

## vanilla-resnet beta-VAE

In [ ]:
class ResBlockBVAE(BVAE):
    """
    simple VAE with a _probably_ pretrained encoder 
    """

    def __init__(self,enc_parts,hidden_dim=None, latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE):
       
        """
        inputs:  
            enc_arch (pre-cut / pretrained)
            enc_dim
            latent_dim
            hidden_dim
            im_size,out_range
        """
        enc_arch,enc_feats,name = enc_parts

        # encoder
        #  arch,cut = xresnet18(pretrained=True),-4
        #  enc_arch = list(arch.children())[:cut]
        
        BASE = im_size//2**5
        enc_dim = enc_feats * BASE**2  # 2**(3*3) * (im_size//32)**2 #(output of resneet) #12800

        self.encoder = build_AE_encoder(enc_arch,enc_dim=enc_dim, hidden_dim=hidden_dim, im_size=im_size)

        in_dim = enc_dim if hidden_dim is None else hidden_dim

        # VAE Bottleneck
        self.bn = VAELayer(in_dim,latent_dim)     

        #decoder
        self.decoder = build_ResBlockAE_decoder(hidden_dim=hidden_dim, latent_dim=latent_dim, im_size=im_size,out_range=out_range)

        store_attr('name,enc_dim, in_dim,hidden_dim,latent_dim,im_size,out_range') # do i need all these?




#  THESE ARE INHERITED..
#     def decode(self, z):    
#         z = self.decoder(z)
#         return z
    
#     def reparam(self, h):
#         return self.bn(h)

#     def encode(self, x):
#         h = self.encoder(x)
#         z, mu, logvar = self.reparam(h)
#         return z, mu, logvar

#     def forward(self, x):
#         z, mu, logvar = self.encode(x)
#         x_hat = self.decode(z)
#         latents = torch.stack([mu,logvar],dim=-1)
#         return x_hat, latents # assume dims are [batch,latent_dim,concat_dim]
        
        
        

### 128 latents, alpha=5

In [ ]:
latent_dim = 128
alpha = 5 # doubled because latent is half?
batchmean = True
useL1 = False

# SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True), 
cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(), ParamScheduler({'kl_weight':  SchedNo(1.,1.) })]


metrics = default_VAE_metrics(alpha,batchmean,useL1)


block = get_ae_DataBlock(aug=True)
batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)

arch='resnblock'
vae = ResBlockBVAE(get_resblockencoder_parts(arch), hidden_dim=2048,latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE)
                   
# let beta be calculated by : 3*im_size*im_size/latent_dim
loss_func = BVAELoss(batchmean=batchmean,alpha=alpha,useL1=False)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split)#.to_fp16() #wd=config['wd'],opt_func=ranger,



In [ ]:
# NOTE: lf_finder does NOT work correctly with our annealed kl_weight... 

lr1,lr2=learn.lr_find()
mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
#put in the annealied KL_weight...
learn.remove_cb(learn.cbs[-1])
# add new constant scheduler
learn.add_cb(ParamScheduler({'kl_weight': default_KL_anneal_in()} ) )

#fit the backend of the VAE (n)
# the defaults are pretty good for now
n_epochs = 10

#learn.fit_one_cycle(freeze_epochs1,lr_max= lr1)#, lr_max= base_lr)
#learn.fit_flat_cos(n_epochs, lr=lr1, pct_start=0.5)
#learn.fit_flat_cos(n_epochs) #, lr=1e-4,pct_start=0.5)
learn.fit_one_cycle(n_epochs)#, lr_max= base_lr)

learn.show_results()


This initial "burning in" of the KLD regularization is very unstable...

In [ ]:
# replace kl_weight scheduler... with a constant unity... (NOTE: should be robust to failure if we don't replace it)
learn.remove_cb(learn.cbs[-1])
# add new constant scheduler
learn.add_cb(ParamScheduler({'kl_weight':  SchedNo(1.,1.) }) )

#learn.unfreeze()


In [ ]:

lr1,lr2=learn.lr_find()

mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
#epochs = 5
base_lr = 1e-5# gmlr #/= 2
epochs = 100

#learn.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div)
#learn.fit_one_cycle(epochs, lr_max= 1e-3)
#learn.fit_flat_cos(epochs,lr=lr1,pct_start=.05)
learn.fit_flat_cos(epochs,div_final=1000)#,lr=1e-4)
learn.show_results()

In [ ]:
prefix = f"BVae-{'2step10_100'}-latent{latent_dim}"
filename = f"{prefix}-{learn.model.name}-alpha{alpha:d}_{datetime.now().strftime('%Y-%m-%d_%H.%M.%S')}"

learn.save(filename)
learn.export(f'{filename}.pkl')

In [ ]:
x = 1

### 128 latents, alpha=10

In [ ]:
latent_dim = 128
alpha = 10 # doubled because latent is half?
batchmean = True
useL1 = False



# SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True), 
cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(), ParamScheduler({'kl_weight':  SchedNo(1.,1.) })]


metrics = default_VAE_metrics(alpha,batchmean,useL1)


block = get_ae_DataBlock(aug=True)
batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)

arch='resnblock'
vae = ResBlockBVAE(get_resblockencoder_parts(arch), hidden_dim=2048,latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE)
                   
# let beta be calculated by : 3*im_size*im_size/latent_dim
loss_func = BVAELoss(batchmean=batchmean,alpha=alpha,useL1=False)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split)#.to_fp16() #wd=config['wd'],opt_func=ranger,



In [ ]:
# NOTE: lf_finder does NOT work correctly with our annealed kl_weight... 

lr1,lr2=learn.lr_find()
mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
#put in the annealied KL_weight...
learn.remove_cb(learn.cbs[-1])
# add new constant scheduler
learn.add_cb(ParamScheduler({'kl_weight': default_KL_anneal_in()} ) )

In [ ]:
# #fit the backend of the VAE (n)
# # the defaults are pretty good for now
n_epochs = 10

learn.fit_one_cycle(n_epochs)#, lr_max= base_lr)
# #learn.fit_flat_cos(n_epochs, lr=lr1, pct_start=0.5)
# learn.fit_flat_cos(n_epochs, lr=1e-4,pct_start=0.5)

# learn.show_results()


In [ ]:
# #fit the backend of the VAE (n)
# # the defaults are pretty good for now
# n_epochs = 10

# learn.fit_one_cycle(10)#, lr_max= base_lr)
# #learn.fit_flat_cos(n_epochs, lr=lr1, pct_start=0.5)
# #learn.fit_flat_cos(n_epochs, lr=1e-4,pct_start=0.5)

# learn.show_results()


This initial "burning in" of the KLD regularization is very unstable...

In [ ]:
# replace kl_weight scheduler... with a constant unity... (NOTE: should be robust to failure if we don't replace it)
learn.remove_cb(learn.cbs[-1])
# add new constant scheduler
learn.add_cb(ParamScheduler({'kl_weight':  SchedNo(1.,1.) }) )

#learn.unfreeze()


In [ ]:

lr1,lr2=learn.lr_find()

mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
#epochs = 5
epochs = 100

#learn.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div)
#learn.fit_one_cycle(epochs, lr_max= 1e-3)
#learn.fit_flat_cos(epochs,lr=lr1,pct_start=.05)
learn.fit_flat_cos(epochs,div_final = 1000)#,lr=1e-4)
learn.show_results()

In [ ]:
learn.lr


In [ ]:
# prefix = f"BVae-{'POST-1CYCLE10'}-latent{latent_dim}"
# filename = f"{prefix}-{learn.model.name}-alpha{alpha:d}_{datetime.now().strftime('%Y-%m-%d_%H.%M.%S')}"

# learn.save(filename)
# learn.export(f'{filename}.pkl')

In [ ]:
# #epochs = 5
# base_lr = 1e-5# gmlr #/= 2
# epochs = 50

# #learn.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div)
# #learn.fit_one_cycle(epochs, lr_max= 1e-3)
# #learn.fit_flat_cos(epochs,lr=lr1,pct_start=.05)
# learn.fit_flat_cos(epochs)
# learn.show_results()

In [ ]:
# prefix = f"BVae-{'POST-1CYCLE10-COS50'}-latent{latent_dim}"
# filename = f"{prefix}-{learn.model.name}-alpha{alpha:d}_{datetime.now().strftime('%Y-%m-%d_%H.%M.%S')}"

# learn.save(filename)
# learn.export(f'{filename}.pkl')

In [ ]:
# prefix = f"BVae-{'POST-1CYCLE10'}-latent{latent_dim}"
# filename = f"{prefix}-{learn.model.name}-alpha{alpha:d}_{datetime.now().strftime('%Y-%m-%d_%H.%M.%S')}"
# filename = "BVae-POST-1CYCLE10-latent128-resblock-alpha10_2021-03-24_21.33.02"
# learn.load(filename)
# #epochs = 5
# epochs = 10
# learn.fit_one_cycle(epochs, lr_max=.001)
# #learn.fit_flat_cos(epochs,lr=.0015,pct_start=.5,div_final=1000.0)
# #learn.fit_one_cycle(epochs,lr_max=5e-3,pct_start=0.5,div_final=100000) # gets down to ~4500 loss in 10
# learn.show_results()

In [ ]:
prefix = f"BVae-{'2step10_100'}-latent{latent_dim}"
filename = f"{prefix}-{learn.model.name}-alpha{alpha:d}_{datetime.now().strftime('%Y-%m-%d_%H.%M.%S')}"

learn.save(filename)
learn.export(f'{filename}.pkl')

### 64 latents, alpha=5

In [ ]:
latent_dim = 64
alpha = 5 # doubled because latent is half?
batchmean = True
useL1 = False


# SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True), 
cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(), ParamScheduler({'kl_weight':  SchedNo(1.,1.) })]


metrics = default_VAE_metrics(alpha,batchmean,useL1)


block = get_ae_DataBlock(aug=True)
batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)

arch='resnblock'
vae = ResBlockBVAE(get_resblockencoder_parts(arch), hidden_dim=2048,latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE)
                   
# let beta be calculated by : 3*im_size*im_size/latent_dim
loss_func = BVAELoss(batchmean=batchmean,alpha=alpha,useL1=False)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split)#.to_fp16() #wd=config['wd'],opt_func=ranger,



In [ ]:
# NOTE: lf_finder does NOT work correctly with our annealed kl_weight... 

lr1,lr2=learn.lr_find()
mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
#put in the annealied KL_weight...
learn.remove_cb(learn.cbs[-1])
# add new constant scheduler
learn.add_cb(ParamScheduler({'kl_weight': default_KL_anneal_in()} ) )

In [ ]:
# the defaults are pretty good for now
n_epochs = 10

learn.fit_one_cycle(n_epochs)#,lr_max= lr1)#, lr_max= base_lr)
#learn.fit_flat_cos(n_epochs, lr=lr1, pct_start=0.5)
#learn.fit_flat_cos(n_epochs, lr=1e-4,pct_start=0.5)

learn.show_results()


This initial "burning in" of the KLD regularization is very unstable...

In [ ]:
# replace kl_weight scheduler... with a constant unity... (NOTE: should be robust to failure if we don't replace it)
learn.remove_cb(learn.cbs[-1])
# add new constant scheduler
learn.add_cb(ParamScheduler({'kl_weight':  SchedNo(1.,1.) }) )

#learn.unfreeze()


In [ ]:

lr1,lr2=learn.lr_find()

mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
#epochs = 5
base_lr = 1e-5# gmlr #/= 2
epochs = 100

#learn.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div)
#learn.fit_one_cycle(epochs, lr_max= 1e-3)
#learn.fit_flat_cos(epochs,lr=lr1,pct_start=.05)
#learn.fit_flat_cos(epochs,lr=1e-4)
learn.fit_flat_cos(epochs, div_final=1000.0)#,lr=1e-3)
learn.show_results()

In [ ]:
prefix = f"BVae-{'2step10_100'}-latent{latent_dim}"
filename = f"{prefix}-{learn.model.name}-alpha{alpha:d}_{datetime.now().strftime('%Y-%m-%d_%H.%M.%S')}"

learn.save(filename)
learn.export(f'{filename}.pkl')

### 64 latents, alpha=10

In [ ]:
latent_dim = 64
alpha = 10 # doubled because latent is half?
batchmean = True
useL1 = False


# SaveModelCallback(fname=datetime.now().strftime('%Y-%m-%d %Hh%M.%S'), every_epoch=True), 
cbs = [AnnealedLossCallback(),TerminateOnNaNCallback(), ParamScheduler({'kl_weight':  SchedNo(1.,1.) })]


metrics = default_VAE_metrics(alpha,batchmean,useL1)


block = get_ae_DataBlock(aug=True)
batch_size = 128
dls = block.dataloaders(df, batch_size=batch_size)

arch='resnblock'
vae = ResBlockBVAE(get_resblockencoder_parts(arch), hidden_dim=2048,latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE)
                   
# let beta be calculated by : 3*im_size*im_size/latent_dim
loss_func = BVAELoss(batchmean=batchmean,alpha=alpha,useL1=False)

learn = Learner(dls, vae, cbs=cbs,loss_func=loss_func, metrics=metrics,splitter=AE_split)#.to_fp16() #wd=config['wd'],opt_func=ranger,



In [ ]:
# NOTE: lf_finder does NOT work correctly with our annealed kl_weight... 

lr1,lr2=learn.lr_find()
mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
#put in the annealied KL_weight...
learn.remove_cb(learn.cbs[-1])
# add new constant scheduler
learn.add_cb(ParamScheduler({'kl_weight': default_KL_anneal_in()} ) )

In [ ]:
#fit the backend of the VAE (n)
# the defaults are pretty good for now
n_epochs = 10

learn.fit_one_cycle(n_epochs)#,lr_max= lr1)#, lr_max= base_lr)
#learn.fit_flat_cos(n_epochs, lr=lr1, pct_start=0.5)
#learn.fit_flat_cos(n_epochs, lr=1e-4,pct_start=0.5)

learn.show_results()


This initial "burning in" of the KLD regularization is very unstable...

In [ ]:
# replace kl_weight scheduler... with a constant unity... (NOTE: should be robust to failure if we don't replace it)
learn.remove_cb(learn.cbs[-1])
# add new constant scheduler
learn.add_cb(ParamScheduler({'kl_weight':  SchedNo(1.,1.) }) )

#learn.unfreeze()


In [ ]:

lr1,lr2=learn.lr_find()

mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
#epochs = 5
base_lr = 1e-5# gmlr #/= 2
epochs = 100

#learn.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div)
#learn.fit_one_cycle(epochs, lr_max= 1e-3)
#learn.fit_flat_cos(epochs,lr=lr1,pct_start=.05)
learn.fit_flat_cos(epochs,div_final=1000.)
learn.show_results()

In [ ]:
prefix = f"BVae-{'2step10_100'}-latent{latent_dim}"
filename = f"{prefix}-{learn.model.name}-alpha{alpha:d}_{datetime.now().strftime('%Y-%m-%d_%H.%M.%S')}"

learn.save(filename)
learn.export(f'{filename}.pkl')

## CODE graveyard 

In [ ]:
# not using...
class LatentTuple(fastuple):
    "Basic type for tuple of tensor (vectors)"
    _show_args = dict(s=10, marker='.', c='r')
    @classmethod
    def create(cls, ts): 
        if isinstance(ts,tuple):
            mu,logvar = ts
        elif ts is None:
            mu,logvar = None,None
        else:
            mu = None
            logvar = None
            
        if mu is None: mu = torch.empty(0)
        elif not isinstance(mu, Tensor): Tensor(mu) 
        
        if logvar is None: logvar = torch.empty(0)
        elif not isinstance(logvar,Tensor): Tensor(logvar)
            
        return cls( (mu,logvar) ) 
        
    def show(self, ctx=None, **kwargs): 
        mu,logvar = self
        if not isinstance(mu, Tensor) or not isinstance(logvar,Tensor): return ctx

        title_str = f"mu-> {mu.mean():e}, {mu.std():e}  logvar->{logvar.mean():e}, {logvar.std():e}"
    
        if 'figsize' in kwargs: del kwargs['figsize']
        if 'title' in kwargs: kwargs['title']=title_str
        if ctx is None:
            _,axs = plt.subplots(1,2, figsize=(12,6))
            x=torch.linspace(0,1,mu[0].shape[0])
            axs[0].scatter(x, mu[:], **{**self._show_args, **kwargs})
            axs[1].scatter(x, logvar[:], **{**self._show_args, **kwargs})
            ctx = axs[1]
        
        ctx.scatter(mu[:], logvar[:], **{**self._show_args, **kwargs})
        return ctx
    
# could we do a typedispatch to manage the transforms...?
def VAETargetTupleBlock(): 
    return TransformBlock(type_tfms=VAETargetTuple.create, batch_tfms=IntToFloatTensor)

def LatentTupleBlock(): 
    return TransformBlock(type_tfms=LatentTuple.create, batch_tfms=noop)
    

# class TensorPoint(TensorBase):
#     "Basic type for points in an image"
#     _show_args = dict(s=10, marker='.', c='r')

#     @classmethod
#     def create(cls, t, img_size=None)->None:
#         "Convert an array or a list of points `t` to a `Tensor`"
#         return cls(tensor(t).view(-1, 2).float(), img_size=img_size)

#     def show(self, ctx=None, **kwargs):
#         if 'figsize' in kwargs: del kwargs['figsize']
#         x = self.view(-1,2)
#         ctx.scatter(x[:, 0], x[:, 1], **{**self._show_args, **kwargs})
#         return ctx



In [ ]:
latent_dim = 128
dropout = .2
im_size = IMG_SIZE
n_blocks = 5        
nfs = [3] + [2**i*n_blocks for i in range(n_blocks+1)] 
nfs.reverse()
# decoder = nn.Sequential(
#             nn.Linear(latent_size, 16),
#             UnFlatten(4),
#             ResBlock(1, 3, 4, act_cls=Mish),
#             nn.Dropout2d(dropout),
#             nn.Upsample(scale_factor=2),
#             ResBlock(1, 4, 8, act_cls=Mish),
#             nn.Dropout2d(dropout),
#             nn.Upsample(scale_factor=2),
#             ResBlock(1, 8, 16, act_cls=Mish),
#             nn.Dropout2d(dropout),
#             nn.Upsample(scale_factor=2),
#             nn.Conv2d(16, 1, 3),
#             nn.Dropout2d(dropout),
#             #nn.AdaptiveAvgPool2d((3,im_size, im_size)) 
# )





n_blocks = 5
hidden_dim = 2048
out_range = [-1,1]
tst = nn.Sequential(
            nn.Linear(latent_dim,hidden_dim), #nn.Linear(latent_dim, 16)
            nn.Linear(hidden_dim,im_size*n_blocks*n_blocks), #nn.Linear(latent_dim, 16)
            ResizeBatch(im_size,n_blocks,n_blocks),#UnFlatten(n_blocks), #4
            ResBlock(1, nfs[0], nfs[1], act_cls=Mish), #ResBlock(1, 1, 4, act_cls=Mish),
            nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, nfs[1], nfs[2], act_cls=Mish), #RResBlock(1, 4, 8, act_cls=Mish),
            nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, nfs[2], nfs[3], act_cls=Mish), #ResBlock(1, 8, 16, act_cls=Mish),
            nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, nfs[3], nfs[4], act_cls=Mish), #nn.Conv2d(16, 1, 3),
            nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, nfs[4], nfs[5], act_cls=Mish), #nn.Conv2d(16, 1, 3),
            nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, nfs[5], nfs[6], act_cls=Mish), #nn.Conv2d(16, 1, 3),
            #nn.Dropout2d(dropout),
            #nn.Upsample(scale_factor=2),            #
            #nn.AdaptiveAvgPool2d((3,im_size, im_size)),
            SigmoidRange(*out_range), #nn.Sigmoid()
)
tst,nfs

In [ ]:
inp=    torch.randn((32,latent_dim))
#last_size = model_sizes(tst ) #[-1][1]
#num_features_model(tst)
#last_size
#nfs
tst(inp).shape,nfs
#last_size

In [ ]:
z_dim = 100
enc = nn.Sequential(
    ResBlock(1, 1, 16, act_cls=nn.ReLU, norm_type=None),
    nn.MaxPool2d(2, 2),
    ResBlock(1, 16, 4, act_cls=nn.ReLU, norm_type=None),
    nn.MaxPool2d(2, 2),
    Flatten()
)
# torch.Size([32, 1, 28, 28])
# torch.Size([32, 16, 28, 28])
# torch.Size([32, 16, 14, 14])
# torch.Size([32, 4, 14, 14])
# torch.Size([32, 4, 7, 7])
# torch.Size([32, 196])

latent_size = 100

enc = nn.Sequential(
            ResBlock(1, 3, 5, stride=2, act_cls=Mish),#  1->3
            ResBlock(1, 5, 5, stride=2, act_cls=Mish),
            ResBlock(1, 5, 1, stride=2, act_cls=Mish),
            Flatten(),
            nn.Linear(400, latent_size) # 16->400
        )
#  torch.Size([32, 1, 28, 28])
# torch.Size([32, 5, 14, 14])
# torch.Size([32, 5, 7, 7])
# torch.Size([32, 1, 4, 4])
# torch.Size([32, 16])
# torch.Size([32, 4])       
inp=    torch.randn((32,3,160,160))
for ii in range(0,8):
    print(enc[:ii](inp).shape)

z = enc(inp)


In [ ]:
dropout=0
dec = nn.Sequential(
            nn.Linear(latent_size, 16),
            UnFlatten(4),
            ResBlock(1, 1, 4, act_cls=Mish), #4->5
            #nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, 4, 8, act_cls=Mish),
            #nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, 8, 16, act_cls=Mish),
            #nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(16, 3, 3), #1->3
            #nn.Dropout2d(dropout),
            nn.AdaptiveAvgPool2d((28, 28)),
            nn.Sigmoid()
        )

# torch.Size([32, 4])
# torch.Size([32, 16])
# torch.Size([32, 1, 4, 4])
# torch.Size([32, 4, 4, 4])
# torch.Size([32, 4, 8, 8])
# torch.Size([32, 8, 8, 8])
# torch.Size([32, 8, 16, 16])
# torch.Size([32, 16, 16, 16])
# torch.Size([32, 16, 32, 32])
# torch.Size([32, 1, 30, 30])
# torch.Size([32, 1, 28, 28])


for ii in range(0,12):
    print(dec[:ii](z).shape)

In [ ]:
n_blocks = 5
BASE = im_size//2**5        
nfs = [3]+[(2**i)*BASE for i in range(n_blocks)] 
n = len(nfs)
hidden_dim = 2048

BASE = im_size//2**5
        # encoder
in_dim = nfs[-1] * BASE**2 
        
modules =  [ResBlock(1, nfs[i],nfs[i+1], 
                          stride=2, act_cls=Mish)  for i in range(n - 1)]    
# enc =  nn.Sequential(
#                 ConvLayer(nfs[0],nfs[1],ks=5,stride=2,padding=2),
#                 ConvLayer(nfs[1],nfs[2],ks=5,stride=2,padding=2),
#                 ConvLayer(nfs[2],nfs[3],ks=5,stride=2,padding=2),
#                 ConvLayer(nfs[3],nfs[4],ks=5,stride=2,padding=2),
#                 ConvLayer(nfs[4],nfs[5],ks=5,stride=2,padding=2),
#                 Flatten(),
#                 LinBnDrop(in_dim,hidden_dim,bn=True,p=0.0,act=nn.ReLU(),lin_first=True)
#             )

enc =  nn.Sequential(*modules,
                Flatten(),
                LinBnDrop(in_dim,hidden_dim,bn=True,p=0.0,act=nn.ReLU(),lin_first=True)
            )

nfs.reverse()

print(nfs)
#last_size = model_sizes(enc, size=(28,28))[-1][1]



    
encoder = nn.Sequential(enc, nn.Linear(hidden_dim, z_dim))
        
decoder = nn.Sequential(
            nn.Linear(z_dim, im_size*n_blocks*n_blocks),
            ResizeBatch(im_size,n_blocks,n_blocks),#UnFlatten(n_blocks), #4
            ResBlock(1, nfs[0], nfs[1], ks=1, act_cls=nn.ReLU, norm_type=None),
            #nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, nfs[1], nfs[2], act_cls=nn.ReLU, norm_type=None),
            #nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(nfs[2], 3, 3, padding=1),
            #nn.Dropout2d(dropout),
            nn.Sigmoid()
        )
#last_size

#             nn.Linear(latent_dim,hidden_dim), #nn.Linear(latent_dim, 16)
#             nn.Linear(hidden_dim,im_size*n_blocks*n_blocks), #nn.Linear(latent_dim, 16)
#             ResizeBatch(im_size,n_blocks,n_blocks),#UnFlatten(n_blocks), #4
#             ResBlock(1, nfs[0], nfs[1], act_cls=Mish), #ResBlock(1, 1, 4, act_cls=Mish),
#             nn.Dropout2d(dropout),
#             nn.Upsample(scale_factor=2),
#             ResBlock(1, nfs[1], nfs[2], act_cls=Mish), #RResBlock(1, 4, 8, act_cls=Mish),
#             nn.Dropout2d(dropout),
#             nn.Upsample(scale_factor=2),
#             ResBlock(1, nfs[2], nfs[3], act_cls=Mish), #ResBlock(1, 8, 16, act_cls=Mish),
#             nn.Dropout2d(dropout),
#             nn.Upsample(scale_factor=2),
#             ResBlock(1, nfs[3], nfs[4], act_cls=Mish), #nn.Conv2d(16, 1, 3),
#             nn.Dropout2d(dropout),
#             nn.Upsample(scale_factor=2),
#             ResBlock(1, nfs[4], nfs[5], act_cls=Mish), #nn.Conv2d(16, 1, 3),
#             nn.Dropout2d(dropout),
#             nn.Upsample(scale_factor=2),
#             ResBlock(1, nfs[5], nfs[6], act_cls=Mish), #nn.Conv2d(16, 1, 3),

            

inp=    torch.randn((32,3,160,160))

#encoder[:1](inp).shape
for ii in range(0,10):
    print(enc[:ii](inp).shape)

z = encoder(inp)
for ii in range(0,14):
    print(decoder[:ii](z).shape)


In [ ]:
class UnFlatten(Module):
    def __init__(self, size=7):
        self.size = size
    def forward(self, input):
        return input.view(input.size(0), -1, self.size, self.size)

class MMD_VAE(Module):
    def __init__(self, latent_size):        
        self.encoder = nn.Sequential(
            ResBlock(1, 1, 5, stride=2, act_cls=Mish),
            ResBlock(1, 5, 5, stride=2, act_cls=Mish),
            ResBlock(1, 5, 1, stride=2, act_cls=Mish),
            Flatten(),
            nn.Linear(16, latent_size)
        )
        
        dropout=0
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, 16),
            UnFlatten(4),
            ResBlock(1, 1, 4, act_cls=Mish),
            nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, 4, 8, act_cls=Mish),
            nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, 8, 16, act_cls=Mish),
            nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(16, 1, 3),
            nn.Dropout2d(dropout),
            nn.AdaptiveAvgPool2d((28, 28)),
            nn.Sigmoid()
        )
        
    
    def forward(self, X):
        latent = self.encoder(X)
        return self.decoder(latent), latent


In [ ]:

    
#decoder
n_blocks = 5        
nfs = [3] + [2**i*n_blocks for i in range(n_blocks+1)] 
nfs.reverse()
n = len(nfs)


tst = nn.Sequential(
            nn.Linear(latent_dim,hidden_dim, #nn.Linear(latent_dim, 16)
            nn.Linear(hidden_dim,im_size*n_blocks*n_blocks) #nn.Linear(latent_dim, 16)
            UnFlatten(n_blocks), #4
            ResBlock(1, nfs[0], nfs[1], act_cls=Mish), #ResBlock(1, 1, 4, act_cls=Mish),
            nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, nfs[1], nfs[2], act_cls=Mish), #RResBlock(1, 4, 8, act_cls=Mish),
            nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, nfs[2], nfs[3], act_cls=Mish), #ResBlock(1, 8, 16, act_cls=Mish),
            nn.Dropout2d(dropout),
            nn.Upsample(scale_factor=2),
            ResBlock(1, nfs[3], nfs[4], act_cls=Mish), #nn.Conv2d(16, 1, 3),
            nn.Dropout2d(dropout),
            nn.AdaptiveAvgPool2d((3,im_size, im_size)),
            SigmoidRange(*out_range)#nn.Sigmoid()

    
                                         *modules,
                                      ConvLayer(nfs[-2],nfs[-1],
                                                ks=1,padding=0, norm_type=None, #act_cls=nn.Sigmoid) )
                                                act_cls=partial(SigmoidRange, *out_range)))
    

In [ ]:
# hide



class AE_vanillaResNet(Module):
    def __init__(self,latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE,pretrained=True):
        #  drop_p=0.0 default turns off dropout
        
        
        self.im_size=im_size
        # encoder

          
        n_blocks = 5
        BASE = IMG_SIZE//2**5
        
        
        nfs = [3]+[(2**i)*BASE for i in range(n_blocks)] 
        n = len(nfs)

        modules =  [ResBlock(1,nfs[i],nfs[i+1],
                                ks=5,stride=2,padding=2) for i in range(n - 1)]       
        #self.in_dim = nfs[-1]*BASE*BASE# Sampling vector
        self.in_dim = nfs[-1]*(BASE)**2# Sampling vector

        self.latent_dim = latent_dim
        self.hidden_dim = 2048
        self.encoder = nn.Sequential(*modules,
                                 Flatten(),
                                 LinBnDrop(self.in_dim,self.hidden_dim,bn=True,p=0.0,act=nn.ReLU(),lin_first=True)
                                )
        

        self.bn = nn.Linear(self.hidden_dim,self.latent_dim)
        #self.bn = AEBottleneck(self.hidden_dim,self.latent_dim)     

        
        nfs = [3] + [2**i*n_blocks for i in range(n_blocks+1)] 
        nfs.reverse()
        n = len(nfs)
        
        modules =  [UpsampleResBlock(nfs[i]) for i in range(n - 2)]        
        self.decoder = nn.Sequential( LinBnDrop(latent_dim,im_size*n_blocks*n_blocks,
                                                bn=True,# batch normalizaiton shouldn't be a problem here
                                                p=0.0,act=nn.ReLU(),lin_first=True),
                                      ResizeBatch(im_size,n_blocks,n_blocks),
                                      *modules,
                                      ConvLayer(nfs[-2],nfs[-1],
                                                ks=1,padding=0, norm_type=None, #act_cls=nn.Sigmoid) )
                                                act_cls=partial(SigmoidRange, *out_range)))
        


    def decode(self, z):    
        return self.decoder(z)
    
    def encode(self, x):
        h = self.encoder(x)
        return self.bn(h)
            
    def forward(self, x):
        """
        pass the "latents" out to keep the learn mechanics consistent... 
        """
        h = self.encoder(x)
        z = self.bn(h)
        x_reconst = self.decoder(z)                    
        dummy_std = torch.ones_like(h) * h.std(dim=1)
        latents = torch.stack([h,torch.exp(0.5 * dummy_std)] ,dim=-1)


        return x_reconst , latents
    

    

In [ ]:
lr1,lr2=learn.lr_find()
mlr = .5*(lr1+lr2)
#geometric mean
gmlr = torch.tensor([lr1,lr2]).log().mean().exp().tolist()

lr1,lr2,mlr,gmlr


In [ ]:
# warm things up...  low-ish learning rate + anealing kldiv retularizaiton into
n_epoch = 10
#learn.fit_flat_cos(n_epoch) #, lr=1e-3, div_final=1e6, pct_start=0.2)
learn.fit_flat_cos(n_epoch, lr=lr1, div_final=1e5, pct_start=0.5)
#learn.fit_one_cycle(n_epoch) #, lr_max= base_lr)

learn.show_results()

In [ ]:
# warm things up...  low-ish learning rate + anealing kldiv retularizaiton into
n_epoch = 40
#learn.unfreeze()
#learn.fit_flat_cos(n_epoch, lr=1e-3, div_final=1e6, pct_start=0.2)
learn.fit_flat_cos(n_epoch, lr=lr1, div_final=1e6, pct_start=0.05)

#learn.fit_flat_cos(n_epoch, lr=1e-3, div_final=1e5, pct_start=0.5)
#learn.fit_one_cycle(n_epoch) #, lr_max= base_lr)

learn.show_results()

In [ ]:
prefix = f"MMDVae-{'bmean' if batchmean else 'mean'}{'l1' if useL1 else 'l2'}"
filename = f"frozen{prefix}-{learn.model.name}-alpha{alpha:d}_{datetime.now().strftime('%Y-%m-%d_%H.%M.%S')}"

learn.save(filename)
learn.export(f'{filename}.pkl')

In [ ]:
##. resblock VAE

In [ ]:
x=1
x

In [ ]:
# hide
    
# check enc_dim... 
class CVAE_vanilla(Module):
    def __init__(self,latent_dim=128, hidden_dim=2048, im_size=IMG_SIZE,out_range=OUT_RANGE,pretrained=True):
        #  drop_p=0.0 default turns off dropout, removed enc_dim=512
        
        self.im_size=im_size
        # encoder

          
        n_blocks = 5
        BASE = im_size//2**5
        
        
        nfs = [3]+[(2**i)*BASE for i in range(n_blocks)] 
        n = len(nfs)

        modules =  [ConvLayer(nfs[i],nfs[i+1],
                                ks=5,stride=2,padding=2) for i in range(n - 1)]       
        #self.in_dim = nfs[-1]*BASE*BASE# Sampling vector
        self.in_dim = nfs[-1]*(BASE)**2# Sampling vector

        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.encoder = nn.Sequential(*modules,
                                 Flatten(),
                                 LinBnDrop(self.in_dim,self.hidden_dim,bn=True,p=0.0,act=nn.ReLU(),lin_first=True)
                                )
        

        self.bn = VAELayer(self.hidden_dim,self.latent_dim)     

        
        nfs = [3] + [2**i*n_blocks for i in range(n_blocks+1)] 
        nfs.reverse()
        n = len(nfs)
        
        modules =  [UpsampleBlock(nfs[i]) for i in range(n - 2)]        
        self.decoder = nn.Sequential( LinBnDrop(latent_dim,self.hidden_dim,
                                                bn=True,# batch normalizaiton shouldn't be a problem here
                                                p=0.0,act=nn.ReLU(),lin_first=True),
                                     LinBnDrop(hidden_dim,im_size*n_blocks*n_blocks,
                                                bn=True,# batch normalizaiton shouldn't be a problem here
                                                p=0.0,act=nn.ReLU(),lin_first=True),
                                      ResizeBatch(im_size,n_blocks,n_blocks),
                                      *modules,
                                      ConvLayer(nfs[-2],nfs[-1],
                                                ks=1,padding=0, norm_type=None, #act_cls=nn.Sigmoid) )
                                                act_cls=partial(SigmoidRange, *out_range)))
        


    def decode(self, z):    
        z = self.decoder(z)
        return z
    
    def reparam(self, h):
        return self.bn(h)

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.reparam(h)
        return z, mu, logvar

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        x_hat = self.decode(z)
        latents = torch.stack([mu,logvar],dim=-1)
        return x_hat, latents # assume dims are [batch,latent_dim,concat_dim]
    
    

    
    

In [ ]:
#hide

# this is the pre-  refactored working version
class VAE_rnet(Module):
    def __init__(self,enc_dim=512, hidden_dim=2048,latent_dim=128, im_size=IMG_SIZE,out_range=OUT_RANGE):
        #  drop_p=0.0 default turns off dropout
        
        self.im_size=im_size
        BASE = im_size//2**5
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        # encoder
        arch,cut = resnet18(pretrained=True),-4
        self.in_dim = enc_dim * BASE**2  # 2**(3*3) * (im_size//32)**2 #(output of resneet) #12800
        
        self.encoder = nn.Sequential(*list(arch.children())[:cut],
                                 Flatten(),
                                 LinBnDrop(self.in_dim,self.hidden_dim,bn=True,p=0.0,act=nn.ReLU(),lin_first=True)
                                )



        # VAE Bottleneck
        self.bn = VAELayer(self.hidden_dim,self.latent_dim)     

        #decoder
        n_blocks = 5        
        nfs = [3] + [2**i*n_blocks for i in range(n_blocks+1)] 
        nfs.reverse()
        n = len(nfs)
        
        modules =  [UpsampleBlock(nfs[i]) for i in range(n - 2)]        
        self.decoder = nn.Sequential( LinBnDrop(latent_dim,self.hidden_dim,
                                                bn=True,# batch normalizaiton shouldn't be a problem here
                                                p=0.0,act=nn.ReLU(),lin_first=True),
                                     LinBnDrop(hidden_dim,im_size*n_blocks*n_blocks,
                                                bn=True,# batch normalizaiton shouldn't be a problem here
                                                p=0.0,act=nn.ReLU(),lin_first=True),
                                      ResizeBatch(im_size,n_blocks,n_blocks),
                                      *modules,
                                      ConvLayer(nfs[-2],nfs[-1],
                                                ks=1,padding=0, norm_type=None, #act_cls=nn.Sigmoid) )
                                                act_cls=partial(SigmoidRange, *out_range)))
        


    def decode(self, z):    
        z = self.decoder(z)
        return z
    
    def reparam(self, h):
        return self.bn(h)

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.reparam(h)
        return z, mu, logvar

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        x_hat = self.decode(z)
        latents = torch.stack([mu,logvar],dim=-1)
        return x_hat, latents # assume dims are [batch,latent_dim,concat_dim]
    
resnet_vae = VAE_rnet(enc_dim=512)
x_hat,latents = resnet_vae.cuda()(x)

In [ ]:
## ARCHITECUTE
def create_encoder(nfs,ks,conv=nn.Conv2d,bn=nn.BatchNorm2d,act_fn = nn.ReLU):
    """
    constructor for generic convolutional encoder 
    """
    n = len(nfs)
    conv_layers = [nn.Sequential(ConvBnRelu(nfs[i],nfs[i+1],kernel_size=ks[i],
                                            conv = conv,bn=bn,act_fn=act_fn, padding = ks[i] //2 ),
                                 Downsample(channels=nfs[i+1],filt_size=3,stride=2))
                                   for i in range(n-1)]        
    convs = nn.Sequential(*conv_layers)
    return convs

def create_encoder_denseblock(n_dense,c_start):
    """
    constructor for resnet with dense blocks  (?) 

    n_dense": 3,
    "c_start": 4
    """
    first_layer = nn.Sequential(ConvBnRelu(3,c_start,kernel_size=3,padding = 1),
                                ResBlock(c_start),
                                Downsample(channels=4,filt_size=3,stride=2))
    
    layers = [first_layer] + [
        nn.Sequential(
            DenseBlock(c_start * (2**c)),
            Downsample(channels=c_start * (2**(c+1)),filt_size=3,stride=2)) for c in range(n_dense)
    ]
    
    model = nn.Sequential(*layers)
    
    return model

def create_decoder(nfs, ks, size, conv=nn.Conv2d, bn=nn.BatchNorm2d, act_fn=nn.ReLU):
    """
    CURR VALUES:
    "nfs":[66,3*32,3*16,3*8,3*4,3*2,3,1,3],
    "ks": [ 3, 1, 3,1,3,1,3,1],   
    "size": IMG_SIZE 
    """
    n = len(nfs)
    
    # We add two channels to the first layer to include x and y channels
    first_layer = ConvBnRelu(nfs[0], #input size 
                             nfs[1], # output size
                             conv = PointwiseConv,
                             bn=bn,
                             act_fn=act_fn)

    conv_layers = [first_layer] + [ConvBnRelu(nfs[i],nfs[i+1],kernel_size=ks[i-1],
                                              padding = ks[i-1] // 2,conv = conv,bn=bn,act_fn=act_fn)
                                   for i in range(1,n - 1)]        
    dec_convs = nn.Sequential(*conv_layers)
    
    dec = nn.Sequential(SpatialDecoder2D(size),dec_convs)
    #SigmoidRange(*y_range)
    return dec

def decoder_simple(y_range=OUT_RANGE, n_out=3):
    return nn.Sequential(#UpsampleBlock(64),
                         UpsampleBlock(32),
                         nn.Conv2d(16, n_out, 1),
                         SigmoidRange(*y_range)
                        )